In [2]:
# libs
%config Completer.use_jedi = False

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import pandas as pd
import os
import datetime
import xarray as xr
from netCDF4 import Dataset
import xesmf as xe
import sys
import warnings
import cmaps
warnings.filterwarnings('ignore')
import glob
import cartopy.feature as cfeature
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.cm as cm
import pytz

# import rum

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl


mpl.rcParams['font.sans-serif'] = "DejaVu Sans"
mpl.rcParams['font.family'] = "sans-serif"
plt.rcParams['figure.figsize'] = (10, 6)
mpl.rc('xtick', labelsize=18)
mpl.rc('ytick', labelsize=18)
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 18

In [3]:
jndir = "/import/GREENING/tzhao/jndata/TROPOMI_HCHO/"
jndirGC = "/import/GREENING/tzhao/jndata/GEOS-Chem/"

jndir_HCHOdiurnal_data = "/import/GREENING/tzhao/jndata/HCHOdiurnal_data/"

ap_gc = xr.open_dataarray(jndirGC+"ap_gc.nc", engine="netcdf4")
b_gc = xr.open_dataarray(jndirGC+"b_gc.nc", engine="netcdf4")
H_b_gc = xr.open_dataarray(jndirGC+"H_b_gc.nc", engine="netcdf4")
P_geoscf = np.array([])

In [4]:
P_geoscf = [
    [1, 0.0100], [2, 0.0200], [3, 0.0327], [4, 0.0476], [5, 0.0660], [6, 0.0893], [7, 0.1197], [8, 0.1595], [9, 0.2113], [10, 0.2785], 
    [11, 0.3650], [12, 0.4758], [13, 0.6168], [14, 0.7951], [15, 1.0194], [16, 1.3005], [17, 1.6508], [18, 2.0850], [19, 2.6202], 
    [20, 3.2764], [21, 4.0766], [22, 5.0468], [23, 6.2168], [24, 7.6198], [25, 9.2929], [26, 11.2769], [27, 13.6434], [28, 16.4571], 
    [29, 19.7916], [30, 23.7304], [31, 28.3678], [32, 33.8100], [33, 40.1754], [34, 47.6439], [35, 56.3879], [36, 66.6034], 
    [37, 78.5123], [38, 92.3657], [39, 108.663], [40, 127.837], [41, 150.393], [42, 176.930], [43, 208.152], [44, 244.875], 
    [45, 288.083], [46, 337.500], [47, 375.000], [48, 412.500], [49, 450.000], [50, 487.500], [51, 525.000], [52, 562.500], 
    [53, 600.000], [54, 637.500], [55, 675.000], [56, 700.000], [57, 725.000], [58, 750.000], [59, 775.000], [60, 800.000], 
    [61, 820.000], [62, 835.000], [63, 850.000], [64, 865.000], [65, 880.000], [66, 895.000], [67, 910.000], [68, 925.000], 
    [69, 940.000], [70, 955.000], [71, 970.000], [72, 985.000]
]

# Convert to numpy array
P_geoscf = np.array(P_geoscf)[36:72,1]

H_geoscf = 153.8 * (20+273.2)* (1- (P_geoscf / 1000) ** 0.1902 )   * 1e-3   # km

In [5]:
def get_directory_list(path):
    # Get a list of all entries in the directory
    entries = os.listdir(path)
    # Filter the list to only include directories
    directories = [e for e in entries if os.path.isdir(os.path.join(path, e))]
    return directories


def count_txt_endlines(filename):
    import codecs
    with codecs.open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()
        num_lines = len(lines)
    return num_lines


def extract_values_from_line(filename, line_number):
    import re

    # Open the file in read-only mode
    with open(filename, 'r', encoding='utf-8', errors='ignore') as file:
        # Read the specified line from the file
        line = file.readlines()[line_number - 1]

        # Use a regular expression to extract the values
        # The regular expression looks for a pattern of one or more non-whitespace characters, followed by ": ", followed by one or more non-whitespace characters
        match = re.search(r'(.*): (.*)', line)
        if match:
            # If the pattern is found, return the two groups of characters
            # If the second group is empty, set the value to "None"
            key = match.group(1)
            value = match.group(2)
            if value:
                return key, value
            else:
                return key, "None"
        else:
            # If the pattern is not found, return None
            return None

def calculate_diurnal_cycle(data, lat, lon, group=None):
    import timezonefinder
    import pytz
    
    # Determine the time zone based on the latitude and longitude
    tf = timezonefinder.TimezoneFinder()
    tz = pytz.timezone(tf.certain_timezone_at(lat=lat, lng=lon))

    hour_LT_list = list( data['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).hour )
    hour_LT_da = xr.DataArray( hour_LT_list, coords=[data.time], dims=['time'])
    diurnal_cycle = data.groupby(hour_LT_da).mean(dim='time')
    if group:
        del(diurnal_cycle)
        diurnal_cycle = data.groupby(hour_LT_da)
    return diurnal_cycle







def calculate_diurnal_cycle_GC(data, lat, lon):
    
    import timezonefinder
    import pytz

    # Determine the time zone based on the latitude and longitude
    tf = timezonefinder.TimezoneFinder()
    tz = pytz.timezone(tf.certain_timezone_at(lat=lat, lng=lon))


    
#     yearlist = np.unique( data.time.dt.year )
    yearlist = np.array([2021,2022,2023])
    yearnum = yearlist.shape[0]
    
    # Create datetime array for 2021
    start_date_2021 = '2021-05-01 00:00:00'
    end_date_2021 = '2021-08-31 00:00:00'
    datetime_array_2021 = pd.date_range(start=start_date_2021, end=end_date_2021, freq='D')
    # Create datetime array for 2022
    start_date_2022 = '2022-05-01 00:00:00'
    end_date_2022 = '2022-08-31 00:00:00'
    datetime_array_2022 = pd.date_range(start=start_date_2022, end=end_date_2022, freq='D')
    # Create datetime array for 2023
    start_date_2023 = '2023-05-01 00:00:00'
    end_date_2023 = '2023-08-31 00:00:00'
    datetime_array_2023 = pd.date_range(start=start_date_2023, end=end_date_2023, freq='D')
    # Combine the two datetime arrays
    datetime_array_combined = datetime_array_2021.union(datetime_array_2022)
    datetime_array_combined = datetime_array_combined.union(datetime_array_2023).copy()
    
    
    
    if len(data.shape)==1:
        data_hour_day = np.full((24, 123*yearnum), np.nan)

        iyear = 0
        for YYYY in yearlist:

            data_YYYY = data[ data.time.dt.year==YYYY ]
            data_YYYY = data_YYYY.where(np.abs(data_YYYY)<1e50 )
            data_YYYY = data_YYYY.resample(time='1H').mean(dim='time').copy()

            hour_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).hour )
            hour_LT_da = xr.DataArray( hour_LT_list, coords=[data_YYYY.time], dims=['time'])
            day_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).dayofyear )
            day_LT_da = xr.DataArray( day_LT_list, coords=[data_YYYY.time], dims=['time'])
            data_YYYY['hour_local'] = hour_LT_da
            data_YYYY['day_local'] = day_LT_da

            # # Group by date and local hour, then aggregate (mean, sum, etc., based on your data)
            grouped = data_YYYY.groupby(day_LT_da)

            for iday in range(121,244):
                try:
                    hour_local = grouped[iday].hour_local.values
                    data_hour_day[:,iday-121+123*iyear][hour_local] = grouped[iday]
                except:
                    print('missed 1 day')
                    continue

            iyear+=1

    if len(data.shape)==2:
        data_hour_day = np.full((24, 123*yearnum, data.shape[1]), np.nan)

        iyear = 0
        for YYYY in yearlist:

            data_YYYY = data[ data.time.dt.year==YYYY ]
            data_YYYY = data_YYYY.where( np.abs(data_YYYY)<1e50 )
            data_YYYY = data_YYYY.resample(time='1H').mean(dim='time').copy()

            hour_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).hour )
            hour_LT_da = xr.DataArray( hour_LT_list, coords=[data_YYYY.time], dims=['time'])
            day_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).dayofyear )
            day_LT_da = xr.DataArray( day_LT_list, coords=[data_YYYY.time], dims=['time'])
            data_YYYY['hour_local'] = hour_LT_da
            data_YYYY['day_local'] = day_LT_da

            # # Group by date and local hour, then aggregate (mean, sum, etc., based on your data)
            grouped = data_YYYY.groupby(day_LT_da)

            for iday in range(121,244):
                try:
                    hour_local = grouped[iday].hour_local.values
                    data_hour_day[:,iday-121+123*iyear,:][hour_local] = grouped[iday]
                except:
                    print('missed 1 day')
                    continue

            iyear+=1


    # Set rows with less than 10 non-NaN values to NaN
#     criteria_hours = 7    # if for a day have < 10 non-nan values, abandon that day
#     data_hour_day[ :, np.sum(~np.isnan(data_hour_day), axis=0) <= criteria_hours] = np.nan

    # Calculate the mean and standard deviation for each hour
    hourly_means = np.nanmedian(data_hour_day, axis=1)
    hourly_stds = np.nanstd(data_hour_day, axis=1)
    data_points_count = np.sum(~np.isnan(data_hour_day), axis=1)

    if len(data.shape)==1:
        diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,24)], dims=['hour_local'])
        diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,24)], dims=['hour_local'])
        diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,24)], dims=['hour_local'])
        diurnal_arrays = xr.DataArray( data_hour_day, coords=[ np.arange(0,24), datetime_array_combined ], dims=['hour_local','day'])
    if len(data.shape)==2:
        if data.shape[1]==12:
            diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,24), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,24), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,24), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_arrays = xr.DataArray( data_hour_day, coords=[np.arange(0,24), datetime_array_combined , PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','day','layer_center_height_bins'])
        elif data.shape[1]==36:
            diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,24), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,24), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,24), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_arrays = xr.DataArray( data_hour_day, coords=[np.arange(0,24), datetime_array_combined , GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','day','lev'])

    return diurnal_means, diurnal_stds, diurnal_amounts, diurnal_arrays






def calculate_diurnal_cycle_GC_cold(data, lat, lon):
    
    import timezonefinder
    import pytz

    # Determine the time zone based on the latitude and longitude
    tf = timezonefinder.TimezoneFinder()
    tz = pytz.timezone(tf.certain_timezone_at(lat=lat, lng=lon))


    
#     yearlist = np.unique( data.time.dt.year )
    yearlist = np.array([2021,2022,2023])
    yearnum = yearlist.shape[0]
    
    # Create datetime array for 2021
    start_date_2021 = '2021-01-01 00:00:00'
    end_date_2021 = '2021-02-28 00:00:00'
    datetime_array_2021 = pd.date_range(start=start_date_2021, end=end_date_2021, freq='D')
    start_date_2021_part2 = '2021-11-01 00:00:00'
    end_date_2021_part2 = '2021-12-31 00:00:00'
    datetime_array_2021_part2 = pd.date_range(start=start_date_2021_part2, end=end_date_2021_part2, freq='D')
    # Combine the two datetime arrays
    datetime_array_2021_combined = datetime_array_2021.union(datetime_array_2021_part2)
    
    
    # Create datetime array for 2022
    start_date_2022 = '2022-01-01 00:00:00'
    end_date_2022 = '2022-02-28 00:00:00'
    datetime_array_2022 = pd.date_range(start=start_date_2022, end=end_date_2022, freq='D')
    start_date_2022_part2 = '2022-11-01 00:00:00'
    end_date_2022_part2 = '2022-12-31 00:00:00'
    datetime_array_2022_part2 = pd.date_range(start=start_date_2022_part2, end=end_date_2022_part2, freq='D')
    # Combine the two datetime arrays
    datetime_array_2022_combined = datetime_array_2022.union(datetime_array_2022_part2)
    
    # Create datetime array for 2023
    start_date_2023 = '2023-01-01 00:00:00'
    end_date_2023 = '2023-02-28 00:00:00'
    datetime_array_2023 = pd.date_range(start=start_date_2023, end=end_date_2023, freq='D')
    start_date_2023_part2 = '2023-11-01 00:00:00'
    end_date_2023_part2 = '2023-12-31 00:00:00'
    datetime_array_2023_part2 = pd.date_range(start=start_date_2023_part2, end=end_date_2023_part2, freq='D')
    # Combine the two datetime arrays
    datetime_array_2023_combined = datetime_array_2023.union(datetime_array_2023_part2)

    
    # Combine the two datetime arrays
    datetime_array_combined = datetime_array_2021_combined.union(datetime_array_2022_combined)
    datetime_array_combined = datetime_array_combined.union(datetime_array_2023_combined).copy()

    
    
    
    if len(data.shape)==1:
        data_hour_day = np.full((24, 120*yearnum), np.nan)

        iyear = 0
        for YYYY in yearlist:
            
            # Generate date ranges for January 1st to February 28th and November 1st to December 31st
            jan_feb_dates = pd.date_range(start=f'{YYYY}-01-01', end=f'{YYYY}-02-28')
            nov_dec_dates = pd.date_range(start=f'{YYYY}-11-01', end=f'{YYYY}-12-31')
            # Extract the day of the year for each date
            jan_feb_dayofyear = jan_feb_dates.dayofyear
            nov_dec_dayofyear = nov_dec_dates.dayofyear
            # Combine the day of the year arrays
            dayofyear_list = np.concatenate([jan_feb_dayofyear, nov_dec_dayofyear])

            # --------  start --------------------------------

            data_YYYY = data[ data.time.dt.year==YYYY ]
            data_YYYY = data_YYYY.where(np.abs(data_YYYY)<1e50 )
            data_YYYY = data_YYYY.resample(time='1H').mean(dim='time').copy()

            hour_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).hour )
            hour_LT_da = xr.DataArray( hour_LT_list, coords=[data_YYYY.time], dims=['time'])
            day_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).dayofyear )
            day_LT_da = xr.DataArray( day_LT_list, coords=[data_YYYY.time], dims=['time'])
            data_YYYY['hour_local'] = hour_LT_da
            data_YYYY['day_local'] = day_LT_da

            # # Group by date and local hour, then aggregate (mean, sum, etc., based on your data)
            grouped = data_YYYY.groupby(day_LT_da)

            for iday in dayofyear_list:
                try:
                    hour_local = grouped[iday].hour_local.values
                    data_hour_day[:, np.where(dayofyear_list==iday)[0][0] + 120*iyear ][hour_local] = grouped[iday]
                except:
                    continue
            iyear+=1

    if len(data.shape)==2:
        data_hour_day = np.full((24, 120*yearnum, data.shape[1]), np.nan)

        iyear = 0
        for YYYY in yearlist:
            
            # Generate date ranges for January 1st to February 28th and November 1st to December 31st
            jan_feb_dates = pd.date_range(start=f'{YYYY}-01-01', end=f'{YYYY}-02-28')
            nov_dec_dates = pd.date_range(start=f'{YYYY}-11-01', end=f'{YYYY}-12-31')
            # Extract the day of the year for each date
            jan_feb_dayofyear = jan_feb_dates.dayofyear
            nov_dec_dayofyear = nov_dec_dates.dayofyear
            # Combine the day of the year arrays
            dayofyear_list = np.concatenate([jan_feb_dayofyear, nov_dec_dayofyear])

            # --------  start --------------------------------

            data_YYYY = data[ data.time.dt.year==YYYY ]
            data_YYYY = data_YYYY.where( np.abs(data_YYYY)<1e50 )
            data_YYYY = data_YYYY.resample(time='1H').mean(dim='time').copy()

            hour_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).hour )
            hour_LT_da = xr.DataArray( hour_LT_list, coords=[data_YYYY.time], dims=['time'])
            day_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).dayofyear )
            day_LT_da = xr.DataArray( day_LT_list, coords=[data_YYYY.time], dims=['time'])
            data_YYYY['hour_local'] = hour_LT_da
            data_YYYY['day_local'] = day_LT_da

            # # Group by date and local hour, then aggregate (mean, sum, etc., based on your data)
            grouped = data_YYYY.groupby(day_LT_da)

            for iday in dayofyear_list:
                try:
                    hour_local = grouped[iday].hour_local.values
                    data_hour_day[:, np.where(dayofyear_list==iday)[0][0] + 120*iyear,:][hour_local] = grouped[iday]
                except:
                    continue
            iyear+=1


    # Set rows with less than 10 non-NaN values to NaN
#     criteria_hours = 7    # if for a day have < 10 non-nan values, abandon that day
#     data_hour_day[ :, np.sum(~np.isnan(data_hour_day), axis=0) <= criteria_hours] = np.nan
    
    # Calculate the mean and standard deviation for each hour
    hourly_means = np.nanmedian(data_hour_day, axis=1)
    hourly_stds = np.nanstd(data_hour_day, axis=1)
    data_points_count = np.sum(~np.isnan(data_hour_day), axis=1)

    if len(data.shape)==1:
        diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,24)], dims=['hour_local'])
        diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,24)], dims=['hour_local'])
        diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,24)], dims=['hour_local'])
        diurnal_arrays = xr.DataArray( data_hour_day, coords=[ np.arange(0,24), datetime_array_combined ], dims=['hour_local','day'])
    if len(data.shape)==2:
        if data.shape[1]==12:
            diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,24), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,24), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,24), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_arrays = xr.DataArray( data_hour_day, coords=[np.arange(0,24), datetime_array_combined , PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','day','layer_center_height_bins'])
        elif data.shape[1]==36:
            diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,24), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,24), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,24), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_arrays = xr.DataArray( data_hour_day, coords=[np.arange(0,24), datetime_array_combined , GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','day','lev'])

    return diurnal_means, diurnal_stds, diurnal_amounts, diurnal_arrays







def find_index_in_strlist(input_str, my_list):
    for i in range(len(my_list)):
        if input_str in my_list[i]:
            return i
    return -1

def sep_sitename_ID(sitename_ID):
    
    import re
    # Your input string
    input_string = sitename_ID
    # Define a regular expression pattern to match the sitename and number
    pattern = r'^(.*?)_P(\d+)$'
    # Use re.search to find the match in the input string
    match = re.search(pattern, input_string)
    
    sitename = match.group(1)
    YY = match.group(2)
    
    return sitename, YY

def lookup_info_fush(info_list, short_sitename):
    for info in info_list:
        if short_sitename in info[2]:
            return info
    return None  # Return None if the short sitename is not found in any sub-list


def ez_cal_diurnal_GC( ds , sitenamelist):
    sitename_datasets = {}
    for sitename in sitenamelist:
        lat = float( lookup_info_fush( info_fush_union, sitename )[0] )
        lon = float( lookup_info_fush( info_fush_union, sitename )[1] )
        ds_diurnal = calculate_diurnal_cycle_GC( ds[sitename], lat, lon )

        # Store the selected data in the dictionary using the sitename as the key
        sitename_datasets[sitename] = ds_diurnal
        
    # Merge all the datasets into a single xarray dataset
    merged_dataset = xr.Dataset(sitename_datasets)
    return merged_dataset


def filter_sites_in_region(info_array, region):
    def is_within_region(lat, lon, region):
        if region =='Global':
            # Define the bounds of North America
            region_bounds = {
                'min_lat': -90.0,    # Minimum latitude
                'max_lat': 90.0,    # Maximum latitude
                'min_lon': -180.0,  # Minimum longitude
                'max_lon': 180.0    # Maximum longitude
            }
        if region =='NA':
            # Define the bounds of North America
            region_bounds = {
                'min_lat': 10.0,    # Minimum latitude
                'max_lat': 80.0,    # Maximum latitude
                'min_lon': -180.0,  # Minimum longitude
                'max_lon': -35.0    # Maximum longitude
#                 'min_lat': 10.0,    # Minimum latitude
#                 'max_lat': 50.0,    # Maximum latitude
#                 'min_lon': -125.0,  # Minimum longitude
#                 'max_lon': -70.0    # Maximum longitude
            }
        elif region =='EUS':
            # Define the bounds of North America
            region_bounds = {
                'min_lat': 30.0,    # Minimum latitude
                'max_lat': 47.0,    # Maximum latitude
                'min_lon': -78.0,  # Minimum longitude
                'max_lon': -60.0    # Maximum longitude
            }
        elif region =='SEUS':
            # Define the bounds of North America
            region_bounds = {
                'min_lat': 25.0,    # Minimum latitude
                'max_lat': 36,    # Maximum latitude
                'min_lon': -90.0,  # Minimum longitude
                'max_lon': -75.0    # Maximum longitude
            }
        elif region =='NEUS':
            # Define the bounds of North America
            region_bounds = {
                'min_lat': 40.0,    # Minimum latitude
                'max_lat': 50,    # Maximum latitude
                'min_lon': -85.0,  # Minimum longitude
                'max_lon': -75.0    # Maximum longitude
            }
        elif region =='SUS':
            # Define the bounds of North America
            region_bounds = {
                'min_lat': 20.0,    # Minimum latitude
#                 'min_lat': 10.0,    # Minimum latitude
                'max_lat': 30,    # Maximum latitude
                'min_lon': -110.0,  # Minimum longitude
                'max_lon': -90.0    # Maximum longitude
            }
        elif region =='MWUS':
            # Define the bounds of North America
            region_bounds = {
                'min_lat': 20.0,    # Minimum latitude
                'max_lat': 50,    # Maximum latitude
                'min_lon': -125.0,  # Minimum longitude
                'max_lon': -105.0    # Maximum longitude
            }
            
            
        elif region == 'EU':
            # Define the bounds of North America
            region_bounds = {
                'min_lat': 25.0,    # Minimum latitude
                'max_lat': 85.0,    # Maximum latitude
                'min_lon': -15.0,   # Minimum longitude
                'max_lon': 50.0     # Maximum longitude
#                 'min_lat': 35.0,    # Minimum latitude
#                 'max_lat': 71.0,    # Maximum latitude
#                 'min_lon': -35.0,   # Minimum longitude
#                 'max_lon': 42.0     # Maximum longitude
                
            }
        elif region == 'AS':
            # Define the bounds of North America
            region_bounds = {
                'min_lat': -10.0,    # Minimum latitude
                'max_lat': 60.0,    # Maximum latitude
                'min_lon': 90.0,  # Minimum longitude
                'max_lon': 180.0    # Maximum longitude
            }
  
        # Check if the coordinates are within the North America bounds
        if (region_bounds['min_lat'] <= lat <= region_bounds['max_lat'] and
            region_bounds['min_lon'] <= lon <= region_bounds['max_lon']):
            return True
        else:
            return False

    # Create a mask to filter sites within ???
    mask = [is_within_region(float(lat), float(lon), region) for lat, lon, _, _, _ in info_array]

    # Use the mask to select sites within North America
    region_sites = info_array[mask]

    return region_sites


# Define a custom function for calculating the mean
def custom_mean(arr, axis=None):

    # Calculate the fraction of NaNs in each variable
    nan_fraction = arr.isnull().mean(dim='group')

    # Filter out variables where more than 50% of values are NaN
    filtered_data_array = arr.sel(variable=nan_fraction <= 0.7)

    # Where the count of non-NaN values is greater than the threshold, calculate the mean
    # Otherwise, return NaN
    return filtered_data_array.mean(dim='variable')


def concat_dataset_2021_2022(ds1, ds2):
    # Step 1: Align the datasets
    # Find the union of all variable names
    all_vars = set(ds1.data_vars).union(set(ds2.data_vars))

    # Add missing variables as NaNs
    for var in all_vars:
        if var not in ds1:
            ds1[var] = xr.full_like(ds1[list(ds1.data_vars)[0]], np.nan)
        if var not in ds2:
            ds2[var] = xr.full_like(ds2[list(ds2.data_vars)[0]], np.nan)

    # Step 3: Concatenate the datasets
    ds12 = xr.concat([ds1, ds2], dim='time')
    return ds12


def relative_amplitude(arr, axis=1):
    # Calculate max, min, and mean along axis=1
    max_values = np.nanmax(arr, axis=axis)
    min_values = np.nanmin(arr, axis=axis)
    mean_values = np.nanmean(arr[12:15], axis=axis)

    # Calculate relative amplitude
    relative_amplitudes = (max_values - min_values) / mean_values
#     relative_amplitudes = (max_values - min_values)
#     relative_amplitudes = (arr)
    return relative_amplitudes

def local_noon(arr, axis=1):
    
    mean_values = np.nanmean(arr[12:15,:], axis=axis)
    return mean_values


def lighten_color(color, amount=0.5):
    import matplotlib.colors as mcolors
    """
    Lightens the given color by mixing it with white.

    Args:
    color (str): The name of the original color.
    amount (float): A value between 0 and 1, where 0 is the original color and 1 is white.

    Returns:
    tuple: The lightened color in RGB.
    """
    # Convert the color name to RGB
    try:
        base_color = mcolors.to_rgb(color)
    except ValueError:
        print(f"Color '{color}' is not recognized. Returning white.")
        return (1, 1, 1)

    # Mix the color with white
    white = np.array([1, 1, 1])
    original = np.array(base_color)
    new_color = original + (white - original) * amount
    return tuple(new_color)


def desaturate_color(color_name, desaturation_factor=0.5):
    """
    Reduce the saturation of a given color name.
    
    :param color_name: The name of the color (e.g., 'red', 'blue')
    :param desaturation_factor: Factor to reduce saturation (between 0 and 1)
    :return: The color with reduced saturation (in RGB format)
    """
    import matplotlib.colors as mcolors
    import colorsys
    
    # Convert the color name to RGB
    rgb = mcolors.to_rgb(color_name)
    
    # Convert RGB to HSL
    h, l, s = colorsys.rgb_to_hls(*rgb)
    
    # Reduce saturation
    s *= desaturation_factor
    
    # Convert HSL back to RGB
    desaturated_rgb = colorsys.hls_to_rgb(h, l, s)
    
    return desaturated_rgb

In [6]:
def calculate_diurnal_cycle_GC_10min(data, lat, lon):
    
    import timezonefinder
    import pytz

    # Determine the time zone based on the latitude and longitude
    tf = timezonefinder.TimezoneFinder()
    tz = pytz.timezone(tf.certain_timezone_at(lat=lat, lng=lon))



#     yearlist = np.unique( data.time.dt.year )
    yearlist = np.array([2021,2022,2023])
    yearnum = yearlist.shape[0]
    
    # Create datetime array for 2021
    start_date_2021 = '2021-05-01 00:00:00'
    end_date_2021 = '2021-08-31 00:00:00'
    datetime_array_2021 = pd.date_range(start=start_date_2021, end=end_date_2021, freq='D')
    # Create datetime array for 2022
    start_date_2022 = '2022-05-01 00:00:00'
    end_date_2022 = '2022-08-31 00:00:00'
    datetime_array_2022 = pd.date_range(start=start_date_2022, end=end_date_2022, freq='D')
    # Create datetime array for 2023
    start_date_2023 = '2023-05-01 00:00:00'
    end_date_2023 = '2023-08-31 00:00:00'
    datetime_array_2023 = pd.date_range(start=start_date_2023, end=end_date_2023, freq='D')
    # Combine the two datetime arrays
    datetime_array_combined = datetime_array_2021.union(datetime_array_2022)
    datetime_array_combined = datetime_array_combined.union(datetime_array_2023).copy()



    if len(data.shape)==1:
        data_hour_day = np.full((144, 123*yearnum), np.nan)

        iyear = 0
        for YYYY in yearlist:
            
            # Generate date ranges for January 1st to February 28th and November 1st to December 31st
            may_aug_dates = pd.date_range(start=f'{YYYY}-05-01', end=f'{YYYY}-08-31')
           # Extract the day of the year for each date
            dayofyear_list = may_aug_dates.dayofyear


            data_YYYY = data[ data.time.dt.year==YYYY ]
            data_YYYY = data_YYYY.where(np.abs(data_YYYY)<1e50 )

            hour_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).hour )
            minute_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).minute // 10 )
            hour_minute_LT_list = [hour+minute/6.0   for hour, minute in zip(hour_LT_list, minute_LT_list)]
            hour_minute_LT_da = xr.DataArray(hour_minute_LT_list, coords=[data_YYYY.time], dims=['time'])
            day_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).dayofyear )
            day_LT_da = xr.DataArray( day_LT_list, coords=[data_YYYY.time], dims=['time'])

            
            data_YYYY['hour_local'] = hour_minute_LT_da
            data_YYYY['day_local'] = day_LT_da
            

        
            # # Group by date and local hour, then aggregate (mean, sum, etc., based on your data)
            grouped = data_YYYY.groupby(day_LT_da)

            for iday in range(121,244):
                try:
                    hourly_data = grouped[iday]
                    if len(hourly_data==144):
                        data_hour_day[:, np.where(dayofyear_list==iday)[0][0] + 120*iyear ] = hourly_data
                except:
                    print('missed 1 day')
                    continue

            iyear+=1

    if len(data.shape)==2:
        data_hour_day = np.full((144, 123*yearnum, data.shape[1]), np.nan)

        iyear = 0
        for YYYY in yearlist:
            
            # Generate date ranges for January 1st to February 28th and November 1st to December 31st
            may_aug_dates = pd.date_range(start=f'{YYYY}-05-01', end=f'{YYYY}-08-31')
           # Extract the day of the year for each date
            dayofyear_list = may_aug_dates.dayofyear

            data_YYYY = data[ data.time.dt.year==YYYY ]
            data_YYYY = data_YYYY.where( np.abs(data_YYYY)<1e50 )

            hour_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).hour )
            minute_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).minute // 10 )
            hour_minute_LT_list = [hour+minute/6.0   for hour, minute in zip(hour_LT_list, minute_LT_list)]
            hour_minute_LT_da = xr.DataArray(hour_minute_LT_list, coords=[data_YYYY.time], dims=['time'])
            day_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).dayofyear )
            day_LT_da = xr.DataArray( day_LT_list, coords=[data_YYYY.time], dims=['time'])
            
            
            data_YYYY['hour_local'] = hour_minute_LT_da
            data_YYYY['day_local'] = day_LT_da

            # # Group by date and local hour, then aggregate (mean, sum, etc., based on your data)
            grouped = data_YYYY.groupby(day_LT_da)

            for iday in range(121,244):
                try:
                    hourly_data = grouped[iday]
                    if len(hourly_data==144):
                        data_hour_day[:, np.where(dayofyear_list==iday)[0][0] + 120*iyear ] = hourly_data
                except:
                    print('missed 1 day')
                    continue

            iyear+=1


    # Set rows with less than 10 non-NaN values to NaN
#     criteria_hours = 7*6    # if for a day have < 10 non-nan values, abandon that day
#     data_hour_day[ :, np.sum(~np.isnan(data_hour_day), axis=0) <= criteria_hours] = np.nan

    # Calculate the mean and standard deviation for each hour
    hourly_means = np.nanmedian(data_hour_day, axis=1)
    hourly_stds = np.nanstd(data_hour_day, axis=1)
    data_points_count = np.sum(~np.isnan(data_hour_day), axis=1)

    if len(data.shape)==1:
        diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,144)], dims=['hour_local'])
        diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,144)], dims=['hour_local'])
        diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,144)], dims=['hour_local'])
        diurnal_arrays = xr.DataArray( data_hour_day, coords=[ np.arange(0,144), datetime_array_combined ], dims=['hour_local','day'])
    if len(data.shape)==2:
        if data.shape[1]==11:
            diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,144), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,144), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,144), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_arrays = xr.DataArray( data_hour_day, coords=[np.arange(0,144), datetime_array_combined , PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','day','layer_center_height_bins'])
        elif data.shape[1]==36:
            diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,144), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,144), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,144), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_arrays = xr.DataArray( data_hour_day, coords=[np.arange(0,144), datetime_array_combined , GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','day','lev'])

    return diurnal_means, diurnal_stds, diurnal_amounts, diurnal_arrays






def calculate_diurnal_cycle_GC_cold_10min(data, lat, lon):
    
    import timezonefinder
    import pytz

    # Determine the time zone based on the latitude and longitude
    tf = timezonefinder.TimezoneFinder()
    tz = pytz.timezone(tf.certain_timezone_at(lat=lat, lng=lon))


    
#     yearlist = np.unique( data.time.dt.year )
    yearlist = np.array([2021,2022,2023])
    yearnum = yearlist.shape[0]
    
    # Create datetime array for 2021
    start_date_2021 = '2021-01-01 00:00:00'
    end_date_2021 = '2021-02-28 00:00:00'
    datetime_array_2021 = pd.date_range(start=start_date_2021, end=end_date_2021, freq='D')
    start_date_2021_part2 = '2021-11-01 00:00:00'
    end_date_2021_part2 = '2021-12-31 00:00:00'
    datetime_array_2021_part2 = pd.date_range(start=start_date_2021_part2, end=end_date_2021_part2, freq='D')
    # Combine the two datetime arrays
    datetime_array_2021_combined = datetime_array_2021.union(datetime_array_2021_part2)
    
    
    # Create datetime array for 2022
    start_date_2022 = '2022-01-01 00:00:00'
    end_date_2022 = '2022-02-28 00:00:00'
    datetime_array_2022 = pd.date_range(start=start_date_2022, end=end_date_2022, freq='D')
    start_date_2022_part2 = '2022-11-01 00:00:00'
    end_date_2022_part2 = '2022-12-31 00:00:00'
    datetime_array_2022_part2 = pd.date_range(start=start_date_2022_part2, end=end_date_2022_part2, freq='D')
    # Combine the two datetime arrays
    datetime_array_2022_combined = datetime_array_2022.union(datetime_array_2022_part2)
    
    # Create datetime array for 2023
    start_date_2023 = '2023-01-01 00:00:00'
    end_date_2023 = '2023-02-28 00:00:00'
    datetime_array_2023 = pd.date_range(start=start_date_2023, end=end_date_2023, freq='D')
    start_date_2023_part2 = '2023-11-01 00:00:00'
    end_date_2023_part2 = '2023-12-31 00:00:00'
    datetime_array_2023_part2 = pd.date_range(start=start_date_2023_part2, end=end_date_2023_part2, freq='D')
    # Combine the two datetime arrays
    datetime_array_2023_combined = datetime_array_2023.union(datetime_array_2023_part2)

    
    # Combine the two datetime arrays
    datetime_array_combined = datetime_array_2021_combined.union(datetime_array_2022_combined)
    datetime_array_combined = datetime_array_combined.union(datetime_array_2023_combined).copy()

    
    
    
    if len(data.shape)==1:
        data_hour_day = np.full((144, 120*yearnum), np.nan)

        iyear = 0
        for YYYY in yearlist:
            
            # Generate date ranges for January 1st to February 28th and November 1st to December 31st
            jan_feb_dates = pd.date_range(start=f'{YYYY}-01-01', end=f'{YYYY}-02-28')
            nov_dec_dates = pd.date_range(start=f'{YYYY}-11-01', end=f'{YYYY}-12-31')
            # Extract the day of the year for each date
            jan_feb_dayofyear = jan_feb_dates.dayofyear
            nov_dec_dayofyear = nov_dec_dates.dayofyear
            # Combine the day of the year arrays
            dayofyear_list = np.concatenate([jan_feb_dayofyear, nov_dec_dayofyear])

            # --------  start --------------------------------

            data_YYYY = data[ data.time.dt.year==YYYY ]
            data_YYYY = data_YYYY.where(np.abs(data_YYYY)<1e50 )

            hour_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).hour )
            minute_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).minute // 10 )
            hour_minute_LT_list = [hour+minute/6.0   for hour, minute in zip(hour_LT_list, minute_LT_list)]
            hour_minute_LT_da = xr.DataArray(hour_minute_LT_list, coords=[data_YYYY.time], dims=['time'])
            day_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).dayofyear )
            day_LT_da = xr.DataArray( day_LT_list, coords=[data_YYYY.time], dims=['time'])

            # # Group by date and local hour, then aggregate (mean, sum, etc., based on your data)
            grouped = data_YYYY.groupby(day_LT_da)

            for iday in dayofyear_list:
                try:
                    hourly_data = grouped[iday]
                    if len(hourly_data==144):
                        data_hour_day[:, np.where(dayofyear_list==iday)[0][0] + 120*iyear ] = hourly_data
                except:
                    continue
            iyear+=1

    if len(data.shape)==2:
        data_hour_day = np.full((144, 120*yearnum, data.shape[1]), np.nan)

        iyear = 0
        for YYYY in yearlist:
            
            # Generate date ranges for January 1st to February 28th and November 1st to December 31st
            jan_feb_dates = pd.date_range(start=f'{YYYY}-01-01', end=f'{YYYY}-02-28')
            nov_dec_dates = pd.date_range(start=f'{YYYY}-11-01', end=f'{YYYY}-12-31')
            # Extract the day of the year for each date
            jan_feb_dayofyear = jan_feb_dates.dayofyear
            nov_dec_dayofyear = nov_dec_dates.dayofyear
            # Combine the day of the year arrays
            dayofyear_list = np.concatenate([jan_feb_dayofyear, nov_dec_dayofyear])

            # --------  start --------------------------------

            data_YYYY = data[ data.time.dt.year==YYYY ]
            data_YYYY = data_YYYY.where( np.abs(data_YYYY)<1e50 )

            hour_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).hour )
            minute_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).minute // 10 )
            hour_minute_LT_list = [hour+minute/6.0   for hour, minute in zip(hour_LT_list, minute_LT_list)]
            hour_minute_LT_da = xr.DataArray(hour_minute_LT_list, coords=[data_YYYY.time], dims=['time'])
            day_LT_list = list( data_YYYY['time'].to_index().tz_localize(pytz.utc).tz_convert(tz).dayofyear )
            day_LT_da = xr.DataArray( day_LT_list, coords=[data_YYYY.time], dims=['time'])

            # # Group by date and local hour, then aggregate (mean, sum, etc., based on your data)
            grouped = data_YYYY.groupby(day_LT_da)

            for iday in dayofyear_list:
                try:
                    hourly_data = grouped[iday]
                    if len(hourly_data==144):
                        data_hour_day[:, np.where(dayofyear_list==iday)[0][0] + 120*iyear ] = hourly_data
                except:
                    continue
            iyear+=1


    # Set rows with less than 10 non-NaN values to NaN
#     criteria_hours = 7    # if for a day have < 10 non-nan values, abandon that day
#     data_hour_day[ :, np.sum(~np.isnan(data_hour_day), axis=0) <= criteria_hours] = np.nan
    
    # Calculate the mean and standard deviation for each hour
    hourly_means = np.nanmedian(data_hour_day, axis=1)
    hourly_stds = np.nanstd(data_hour_day, axis=1)
    data_points_count = np.sum(~np.isnan(data_hour_day), axis=1)

    if len(data.shape)==1:
        diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,144)], dims=['hour_local'])
        diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,144)], dims=['hour_local'])
        diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,144)], dims=['hour_local'])
        diurnal_arrays = xr.DataArray( data_hour_day, coords=[ np.arange(0,144), datetime_array_combined ], dims=['hour_local','day'])
    if len(data.shape)==2:
        if data.shape[1]==10:
            diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,144), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,144), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,144), PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','layer_center_height_bins'])
            diurnal_arrays = xr.DataArray( data_hour_day, coords=[np.arange(0,144), datetime_array_combined , PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values], dims=['hour_local','day','layer_center_height_bins'])
        elif data.shape[1]==36:
            diurnal_means = xr.DataArray( hourly_means, coords=[np.arange(0,144), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_stds = xr.DataArray( hourly_stds, coords=[np.arange(0,144), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_amounts = xr.DataArray( data_points_count, coords=[np.arange(0,144), GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','lev'])
            diurnal_arrays = xr.DataArray( data_hour_day, coords=[np.arange(0,144), datetime_array_combined , GEOSCF_HCHO_v36_2021_warm.lev], dims=['hour_local','day','lev'])

    return diurnal_means, diurnal_stds, diurnal_amounts, diurnal_arrays

In [7]:
import ee

# Initialize Earth Engine
ee.Initialize()

def classify_area(latitude, longitude, radius_km):
    # Define the center point as a geometry
    center_point = ee.Geometry.Point(longitude, latitude)

    # Define a circular region of interest (ROI) around the center point
    roi = center_point.buffer(radius_km * 1000)  # Convert radius from km to meters

    # Load the land cover dataset (Copernicus Global Land Cover)
    land_cover = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019')\
        .select('urban-coverfraction')  # Select the 'urban-coverfraction' band

    # Clip the land cover dataset to the ROI
    clipped_land_cover = land_cover.clip(roi)

    # Get the mean value of the 'urban-coverfraction' band within the circular ROI
    mean_cover_fraction = clipped_land_cover.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=roi,
        scale=100
    ).get('urban-coverfraction')

    # Evaluate the server-side computation to get the actual result
    mean_cover_fraction = mean_cover_fraction.getInfo()

    # Classify as 'Urban' if the mean cover fraction exceeds a certain threshold (50), otherwise 'Rural'
    if mean_cover_fraction >= 50:
        land_class = 'Urban'
    else:
        land_class = 'Rural'

    return land_class

# # Example usage
# latitude = 64  # Example latitude
# longitude = -147.0060  # Example longitude
# radius_km = 10  # Example radius in kilometers
# classification = classify_area(latitude, longitude, radius_km)
# print('Area classification:', classification)



# Load the population density image
population_density = ee.ImageCollection('CIESIN/GPWv411/GPW_Population_Density').first()

# Define a function to calculate population within a radius circle
def calculate_population(lat, lon, radius_meters):
    # Create a point geometry
    point = ee.Geometry.Point(lon, lat)
    
    # Buffer the point to create a circle with the specified radius
    circle = point.buffer(radius_meters)
    
    # Clip the population density image to the circle
    population_density_clip = population_density.clip(circle)
    
    # Calculate the total population within the circle
    population_stats = population_density_clip.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=circle,
        scale=1000  # Resolution in meters
    )
    
    # Extract the population count from the result
    population_count = population_stats.get('population_density')
    
    return population_count.getInfo()


# # Example usage: Calculate population within a 1 km radius circle at a specific location
# latitude = 40.730610  # Example latitude (New York City)
# longitude = -73.935242  # Example longitude (New York City)
# radius_meters = 1000  # 1 km radius

# population_within_circle = calculate_population(latitude, longitude, radius_meters)
# print("Population within the circle:", population_within_circle)

In [8]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.metrics import mean_squared_error
from statsmodels.api import OLS, add_constant



fontnum = 22

from pylib.smafit import smafit
import statsmodels.formula.api as smf
def LR_SMA(x,y):
    import xarray as xr
    """
    Input two xarray timeseries (x and y),
    first do an inner aligh, then use SMA fitting to get the slope and intercept
    
    x,y: xarray 1D array
        
    """
#     x_align, y_align = xr.align( x, y, join="inner")
    x_align, y_align = x, y
    s,i,stds,stdi,cis,cii = smafit( x_align, y_align, cl=0.95,robust=True)
    return s,i


def binscatter(x,y,bins):
    import pandas as pd

    df = pd.DataFrame({'X':x, 'Y':y})
    my_bins = pd.cut(x, bins=bins)
    data = df[['X', 'Y']].groupby(my_bins).agg(['mean', 'median', 'size','std'])
#     data.plot.scatter(x=('x', 'mean'), y=('y', 'mean'))
    return data



def add_text(ax, x, y, x_position=0.05, color=None):
    
    # Remove NaN values
    valid_data = ~np.isnan(x) & ~np.isnan(y)
    x_clean, y_clean = x[valid_data], y[valid_data]

    # Perform SMA regression
#     slope_sma, intercept_sma = LR_SMA(x_clean, y_clean)
    slope_sma, _, _, _ = np.linalg.lstsq(x_clean[:,np.newaxis], y_clean[:,np.newaxis]);
    slope_sma = slope_sma.squeeze()
    intercept_sma = np.zeros((slope_sma.shape))
    
    print( slope_sma )
    
    # Calculate correlation coefficient
    r = np.corrcoef(x_clean, y_clean)[0, 1]
    rmse = np.sqrt(np.mean((y_clean - (slope_sma * x_clean + intercept_sma))**2))

#     sma_equation = f"Y = {slope_sma:.2f}X + {intercept_sma:.2f}"
    sma_equation = f"Y = {slope_sma:.2f}X"
    n_points = f"N = {len(x_clean)}"
    rmse_text = f"RMSE = {rmse:.2f}"
    
    # Add text to the top left corner of each panel
    ax.text(x_position, 0.95, sma_equation, transform=ax.transAxes, fontsize=20, verticalalignment='top', color=color)
    ax.text(x_position, 0.88, n_points, transform=ax.transAxes, fontsize=20, verticalalignment='top', color=color)
    ax.text(x_position, 0.81, f"R = {r:.3f}", transform=ax.transAxes, fontsize=20, verticalalignment='top', color=color)
    ax.text(x_position, 0.73, rmse_text, transform=ax.transAxes, fontsize=20, verticalalignment='top', color=color)

    # Plot SMA regression line
    line_color = color if color is not None else 'k'
    ax.plot(x_clean, slope_sma * x_clean + intercept_sma,\
            linewidth=2, color=line_color, linestyle='-', zorder=10)



def remove_outliers(x_values, y_values):
    # Convert to numpy arrays if they aren't already
    x = np.array(x_values)
    y = np.array(y_values)

    # Calculate Q1 and Q3 for y_values
    Q1 = np.nanpercentile(y, 25)
    Q3 = np.nanpercentile(y, 75)

    # Calculate the IQR
    IQR = Q3 - Q1

    # Define the bounds for outliers
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    print(Q1)

    # Identify outlier indices
    outlier_indices = (y < lower_bound) | (y > upper_bound)

    # Set outliers to NaN
    x[outlier_indices] = np.nan
    y[outlier_indices] = np.nan

    return x, y


def LR_zerointercept(x,y):
    # Remove NaN values
    valid_data = ~np.isnan(x) & ~np.isnan(y)
    x_clean, y_clean = x[valid_data], y[valid_data]

    # Perform SMA regression
    slope, _, _, _ = np.linalg.lstsq(x_clean[:,np.newaxis], y_clean[:,np.newaxis]);
    slope = float(slope.squeeze())
    
    return slope


def desaturate_color(color_name, desaturation_factor=0.5):
    """
    Reduce the saturation of a given color name.
    
    :param color_name: The name of the color (e.g., 'red', 'blue')
    :param desaturation_factor: Factor to reduce saturation (between 0 and 1)
    :return: The color with reduced saturation (in RGB format)
    """
    import matplotlib.colors as mcolors
    import colorsys
    
    # Convert the color name to RGB
    rgb = mcolors.to_rgb(color_name)
    
    # Convert RGB to HSL
    h, l, s = colorsys.rgb_to_hls(*rgb)
    
    # Reduce saturation
    s *= desaturation_factor
    
    # Convert HSL back to RGB
    desaturated_rgb = colorsys.hls_to_rgb(h, l, s)
    
    return desaturated_rgb

def fill_vars(ds1, namestr):
    
    variableslist = info_fush_union[:,2]
    # Items to remove
    items_to_remove = {'time', 'lev', 'layer_center_height_bins'}
    # New list excluding the items to remove
    filtered_variableslist = [var for var in variableslist if var not in items_to_remove]
    
    if namestr == 'sitename_PID':
        ds2 = ds1.copy()
        # Add missing variables as NaNs
        for var in filtered_variableslist:
            if var not in list(ds2.variables):
                ds2[var] = xr.full_like(ds2[list(ds2.data_vars)[0]], np.nan)

    if namestr == 'sitename':
        ds2 = {}
        for var in filtered_variableslist:
            if sep_sitename_ID(var)[0] in list(ds1.variables):
                ds2[var] = ds1[ sep_sitename_ID(var)[0] ]
            if sep_sitename_ID(var)[0] not in list(ds1.variables):
                ds2[var] = xr.full_like(ds1[list(ds1.data_vars)[0]], np.nan)        
        
        ds2 = xr.Dataset(ds2)
        
    return ds2

In [9]:
# New PGN data, more sites available!  ( Globally     107    sites that have both fuh and fus data in all time periods)

info_fush_union = np.array([['-0.2046', '100.3195', 'Agam_P211', '865.0'],
       ['-27.3493', '30.1438', 'Wakkerstroom_P159', '1760.0'],
       ['-45.7833', '-67.45', 'ComodoroRivadavia_P124', '46.0'],
       ['-6.8948', '107.5865', 'Bandung_P210', '752.0'],
       ['1.299', '103.771', 'Singapore-NUS_P77', '77.0'],
       ['13.7847', '100.54', 'Bangkok_P190', '60.0'],
       ['18.3797', '-65.6184', 'Fajardo_P60', '66.0'],
       ['19.1187', '-98.6552', 'Altzomoni_P65', '3985.0'],
       ['19.3262', '-99.1761', 'MexicoCity-UNAM_P142', '2280.0'],
       ['19.483', '-99.147', 'MexicoCity-Vallejo_P157', '2255.0'],
       ['23.7284', '90.3982', 'Dhaka_P76', '34.0'],
       ['28.309', '-16.4994', 'Izana_P121', '2360.0'],
       ['28.309', '-16.4994', 'Izana_P209', '2360.0'],
       ['29.6721', '-95.0647', 'LaPorteTX_P11', '22.0'],
       ['29.6721', '-95.0647', 'LaPorteTX_P58', '22.0'],
       ['29.6721', '-95.0647', 'LaPorteTX_P63', '22.0'],
       ['29.72', '-95.34', 'HoustonTX_P25', '19.0'],
       ['29.9011', '-95.3262', 'AldineTX_P61', '8.0'],
       ['30.0965', '-94.7635', 'LibertyTX_P143', '3.0'],
       ['32.1129', '34.8062', 'Tel-Aviv_P182', '76.0'],
       ['32.7316', '-97.1142', 'ArlingtonTX_P207', '15.0'],
       ['33.5491', '130.366', 'Fukuoka_P199', '55.0'],
       ['33.6479', '72.9896', 'Islamabad-NUST_P73', '555.0'],
       ['33.779', '-84.3958', 'AtlantaGA-SouthDeKalb_P237', '310.0'],
       ['33.779', '-84.3958', 'AtlantaGA_P173', '310.0'],
       ['34.3819', '-117.6813', 'WrightwoodCA_P68', '2207.0'],
       ['34.719', '135.29', 'Kobe_P198', '23.0'],
       ['34.7252', '-86.6464', 'HuntsvilleAL_P66', '221.0'],
       ['34.96', '-117.8811', 'EdwardsCA_P74', '692.0'],
       ['35.1517', '136.9721', 'Nagoya_P197', '117.0'],
       ['35.2353', '129.0825', 'Busan_P20', '71.0'],
       ['35.3207', '139.6508', 'Yokosuka_P146', '5.0'],
       ['35.5745', '129.1896', 'Ulsan_P150', '38.0'],
       ['35.62', '139.3834', 'Tokyo-TMU_P194', '135.0'],
       ['35.9708', '-79.0933', 'ChapelHillNC_P70', '50.0'],
       ['36.0506', '140.1202', 'Tsukuba-NIES-West_P163', '30.0'],
       ['36.0513', '140.121', 'Tsukuba-NIES_P176', '45.0'],
       ['36.0661', '140.1244', 'Tsukuba_P193', '51.0'],
       ['36.7769', '126.4938', 'Seosan_P164', '25.0'],
       ['37.0203', '-76.3366', 'HamptonVA-HU_P156', '19.0'],
       ['37.164', '-3.605', 'Granada_P238', '680.0'],
       ['37.326', '-77.2057', 'CharlesCityVA_P31', '6.0'],
       ['37.3325', '-121.8821', 'SanJoseCA_P181', '69.0'],
       ['37.42', '-122.0568', 'MountainViewCA_P34', '50.0'],
       ['37.458', '126.951', 'Seoul-SNU_P149', '116.0'],
       ['37.5644', '126.934', 'Seoul_P54', '86.0'],
       ['37.5689', '126.6375', 'Incheon-ESC_P189', '6.0'],
       ['37.8439', '-75.4775', 'WallopsIslandVA_P40', '11.0'],
       ['37.913', '-122.336', 'RichmondCA_P52', '5.0'],
       ['37.9878', '23.775', 'Athens-NOA_P119', '130.0'],
       ['38.9218', '-77.0124', 'WashingtonDC_P140', '58.0'],
       ['38.9926', '-76.8396', 'GreenbeltMD_P2', '90.0'],
       ['38.9926', '-76.8396', 'GreenbeltMD_P32', '90.0'],
       ['39.0553', '-76.8783', 'BeltsvilleMD_P80', '73.0'],
       ['39.1022', '-96.6096', 'ManhattanKS_P165', '346.0'],
       ['39.99', '-105.26', 'BoulderCO_P57', '1660.0'],
       ['39.9919', '-75.0811', 'PhiladelphiaPA_P166', '6.0'],
       ['40.0048', '116.3786', 'Beijing-RADI_P171', '59.0'],
       ['40.0375', '-105.242', 'BoulderCO-NCAR_P204', '1616.0'],
       ['40.1074', '-74.8824', 'BristolPA_P134', '10.0'],
       ['40.4622', '-74.4294', 'NewBrunswickNJ_P69', '19.0'],
       ['40.4655', '-79.9608', 'PittsburghPA_P187', '265.0'],
       ['40.548', '-112.07', 'SouthJordanUT_P139', '1582.0'],
       ['40.6336', '22.9561', 'Thessaloniki_P240', '60.0'],
       ['40.6703', '-74.1261', 'BayonneNJ_P38', '3.0'],
       ['40.7344', '-111.8722', 'SaltLakeCityUT-Hawthorne_P72', '1306.0'],
       ['40.7361', '-73.8215', 'QueensNY_P55', '25.0'],
       ['40.7663', '-111.8478', 'SaltLakeCityUT_P154', '1455.0'],
       ['40.8153', '-73.9505', 'ManhattanNY-CCNY_P135', '34.0'],
       ['40.8679', '-73.8781', 'BronxNY_P180', '31.0'],
       ['40.9635', '-73.1402', 'OldFieldNY_P51', '3.0'],
       ['41.1183', '-73.3367', 'WestportCT_P177', '4.0'],
       ['41.2568', '-72.5533', 'MadisonCT_P186', '3.0'],
       ['41.3014', '-72.9029', 'NewHavenCT_P64', '4.0'],
       ['41.3758', '-72.1004', 'NewLondonCT_P236', '30.0'],
       ['41.8213', '-73.2973', 'CornwallCT_P179', '505.0'],
       ['41.8403', '12.6475', 'Rome-ISAC_P115', '117.0'],
       ['41.841', '-71.361', 'EastProvidenceRI_P185', '15.0'],
       ['41.9017', '12.5158', 'Rome-SAP_P117', '75.0'],
       ['41.9017', '12.5158', 'Rome-SAP_P138', '75.0'],
       ['42.1057', '12.6402', 'Rome-IIA_P138', '92.0'],
       ['42.2929', '-83.0731', 'Windsor-West_P208', '180.0'],
       ['42.3026', '-83.1068', 'SWDetroitMI_P147', '178.0'],
       ['42.3067', '-83.1488', 'DearbornMI_P39', '181.0'],
       ['42.38', '-71.11', 'CambridgeMA_P26', '60.0'],
       ['42.4746', '-70.9708', 'LynnMA_P153', '52.0'],
       ['42.8625', '-71.3801', 'LondonderryNH_P183', '108.0'],
       ['43.0727', '141.3459', 'Sapporo_P196', '46.0'],
       ['43.561', '-70.2073', 'CapeElizabethME_P184', '24.0'],
       ['43.5773', '104.4191', 'Dalanzadgad_P217', '1466.0'],
       ['43.7094', '-79.5435', 'Toronto-West_P108', '141.0'],
       ['43.781', '-79.468', 'Downsview_P104', '187.0'],
       ['43.781', '-79.468', 'Downsview_P170', '187.0'],
       ['43.7843', '-79.1874', 'Toronto-Scarborough_P145', '137.0'],
       ['44.23', '-79.78', 'Egbert_P108', '251.0'],
       ['46.8', '9.83', 'Davos_P120', '1590.0'],
       ['47.2643', '11.3852', 'Innsbruck_P106', '616.0'],
       ['47.2643', '11.3852', 'Innsbruck_P110', '616.0'],
       ['47.9188', '106.848', 'Ulaanbaatar_P216', '1305.0'],
       ['50.798', '4.358', 'Brussels-Uccle_P162', '107.0'],
       ['50.908', '6.413', 'Juelich_P30', '94.0'],
       ['50.9389', '6.9787', 'Cologne_P67', '50.0'],
       ['53.0813', '8.8126', 'Bremen_P21', '50.0'],
       ['60.2037', '24.9612', 'Helsinki_P105', '97.0'],
       ['64.8594', '-147.8499', 'FairbanksAK_P174', '227.0'],
       ['7.342', '134.4722', 'Palau_P131', '23.0'],
       ['78.9233', '11.9299', 'NyAlesund_P152', '18.0']], dtype='<U32')

info_fush_union = np.append(info_fush_union, np.empty((len(info_fush_union), 1), dtype=object), axis=1)

In [10]:
info_fush_union[:, 4] = ""
radius_km = 5

#  reprocess info array if needed
for i, site_info in enumerate(info_fush_union):
    lat = float(site_info[0])
    lon = float(site_info[1])
    landclass = classify_area(lat, lon, radius_km)
    info_fush_union[i, 4] += landclass


    
urban_rows = info_fush_union[(info_fush_union[:, 4] == 'Urban') ]
rural_rows = info_fush_union[(info_fush_union[:, 4] == 'Rural') ]

urban_rows_NA = filter_sites_in_region( urban_rows , "NA" )
rural_rows_NA = filter_sites_in_region( rural_rows , "NA" )
urbanrural_rows_NA = filter_sites_in_region( info_fush_union , "NA" )

urban_rows_EU = filter_sites_in_region( urban_rows , "EU" )
rural_rows_EU = filter_sites_in_region( rural_rows , "EU" )
urbanrural_rows_EU = filter_sites_in_region( info_fush_union , "EU" )

urban_rows_AS = filter_sites_in_region( urban_rows , "AS" )
rural_rows_AS = filter_sites_in_region( rural_rows , "AS" )
urbanrural_rows_AS = filter_sites_in_region( info_fush_union , "AS" )

urban_rows_Global = filter_sites_in_region( urban_rows , "Global" )
rural_rows_Global = filter_sites_in_region( rural_rows , "Global" )
urbanrural_rows_Global = filter_sites_in_region( info_fush_union , "Global" )

#   reprocess info array if needed
for i, site_info in enumerate(info_fush_union):
    site_name = site_info[2]
    if any(urbanrural_rows_AS[:, 2] == site_name):
        info_fush_union[i, 4] += " AS"
    elif any(urbanrural_rows_EU[:, 2] == site_name):
        info_fush_union[i, 4] += " EU"
    elif any(urbanrural_rows_NA[:, 2] == site_name):
        info_fush_union[i, 4] += " NA" 


In [11]:
#  detailed regions of NA

urban_rows_EUS = filter_sites_in_region( urban_rows , "EUS" )
rural_rows_EUS = filter_sites_in_region( rural_rows , "EUS" )
urbanrural_rows_EUS = filter_sites_in_region( info_fush_union , "EUS" )

urban_rows_SEUS = filter_sites_in_region( urban_rows , "SEUS" )
rural_rows_SEUS = filter_sites_in_region( rural_rows , "SEUS" )
urbanrural_rows_SEUS = filter_sites_in_region( info_fush_union , "SEUS" )

urban_rows_NEUS = filter_sites_in_region( urban_rows , "NEUS" )
rural_rows_NEUS = filter_sites_in_region( rural_rows , "NEUS" )
urbanrural_rows_NEUS = filter_sites_in_region( info_fush_union , "NEUS" )

urban_rows_SUS = filter_sites_in_region( urban_rows , "SUS" )
rural_rows_SUS = filter_sites_in_region( rural_rows , "SUS" )
urbanrural_rows_SUS = filter_sites_in_region( info_fush_union , "SUS" )

urban_rows_MWUS = filter_sites_in_region( urban_rows , "MWUS" )
rural_rows_MWUS = filter_sites_in_region( rural_rows , "MWUS" )
urbanrural_rows_MWUS = filter_sites_in_region( info_fush_union , "MWUS" )


In [12]:
# add population density (people per km2)
if info_fush_union.shape[1] <= 5:
    info_fush_union = np.insert(info_fush_union, 5, values="", axis=1)
    
radius_meter = 5*1000     # function uses meter

#  reprocess info array if needed
for i, site_info in enumerate(info_fush_union):
    lat = float(site_info[0])
    lon = float(site_info[1])
    populationdensity = calculate_population(lat, lon, radius_meter)
    info_fush_union[i, 5] += str(populationdensity)
    info_fush_union[i, 5] = float(info_fush_union[i, 5])

In [13]:
# Convert info_fush_union to dataframe

info_fush_union_df = pd.DataFrame(info_fush_union)
column_names = ['lat', 'lon', 'sitename_PID', 'alt_m', 'landtype_region','population_5kmcircle']
# Convert to DataFrame with column names
info_fush_union_df = pd.DataFrame(info_fush_union, columns=column_names)

# Convert specific columns from string to float
columns_to_convert = ['lat', 'lon', 'alt_m', 'population_5kmcircle']
for col_idx in columns_to_convert:
    info_fush_union_df[col_idx] = info_fush_union_df[col_idx].astype(float)

In [64]:
# load GEOS-CF data




def load_geoscf_data_csv(year, start_date, end_date):
    folder_path = '/import/GREENING/tzhao/GEOSCF_subset/'
    prefix = f'GEOSCF'
    suffix = f'{year}-{start_date}_{year}-{end_date}.csv'

    hcho_data = pd.read_csv(folder_path + prefix + '_HCHO_' + suffix, sep=',', index_col=0)
    hcho_data.index = pd.to_datetime(hcho_data.index)
    hcho_data = hcho_data.drop_duplicates().to_xarray()
    hcho_data = hcho_data.rename({'datetime_UTC': 'time'})

    tropcol_hcho_data = pd.read_csv(folder_path + prefix + '_TROPCOL_HCHO_' + suffix, sep=',', index_col=0)
    tropcol_hcho_data.index = pd.to_datetime(tropcol_hcho_data.index)
    tropcol_hcho_data = tropcol_hcho_data.drop_duplicates().to_xarray()
    tropcol_hcho_data = tropcol_hcho_data.rename({'datetime_UTC': 'time'})

    totcol_hcho_data = pd.read_csv(folder_path + prefix + '_TOTCOL_HCHO_' + suffix, sep=',', index_col=0)
    totcol_hcho_data.index = pd.to_datetime(totcol_hcho_data.index)
    totcol_hcho_data = totcol_hcho_data.drop_duplicates().to_xarray()
    totcol_hcho_data = totcol_hcho_data.rename({'datetime_UTC': 'time'})

    return hcho_data, tropcol_hcho_data, totcol_hcho_data

GEOSCF_HCHO_2021_warm, GEOSCF_TROPCOL_HCHO_2021_warm, GEOSCF_TOTCOL_HCHO_2021_warm = load_geoscf_data_csv('2021', '05-01', '08-31')
GEOSCF_HCHO_2022_warm, GEOSCF_TROPCOL_HCHO_2022_warm, GEOSCF_TOTCOL_HCHO_2022_warm = load_geoscf_data_csv('2022', '05-01', '08-31')
GEOSCF_HCHO_2023_warm, GEOSCF_TROPCOL_HCHO_2023_warm, GEOSCF_TOTCOL_HCHO_2023_warm = load_geoscf_data_csv('2023', '05-01', '08-31')


In [65]:
def load_geoscf_vp_csv(year, start_date, end_date):
    """
    Read vertical profile *.csv files from a directory, concatenate them,
    and load into a netCDF dataset.

    Parameters:
    year (int): Year for which to load data.
    start_date (str): Start date of the time range (format: MM-DD).
    end_date (str): End date of the time range (format: MM-DD).

    Returns:
    xarray.Dataset: NetCDF dataset containing concatenated data.
    """

    directory = '/import/GREENING/tzhao/GEOSCF_subset_opendap/'
    all_dataframes = []

    start_date = pd.to_datetime(f"{year}-{start_date}")
    end_date = pd.to_datetime(f"{year}-{end_date}")

    for filename in os.listdir(directory):
        if 'chm_tavg_1hr_g1440x721_v36__HCHO_merged_' + str(year) in filename and filename.endswith('.csv'):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['time'] = pd.to_datetime(df['time'])
            # Set minute, second, and microsecond to zero
            df['time'] = df['time'].dt.floor('H')  # Set to the beginning of the hour
            df = df[(df['time'] >= start_date) & (df['time'] <= end_date)]
            all_dataframes.append(df)

    concatenated_df = pd.concat(all_dataframes)
    concatenated_df = concatenated_df.drop_duplicates(subset=['time', 'lev'], keep='first')
    concatenated_df['time'] = pd.to_datetime(concatenated_df['time'])
    
    xarray_dataset = xr.Dataset.from_dataframe(concatenated_df.set_index(['time', 'lev']))
    xarray_dataset = xarray_dataset.assign_coords(lev=H_geoscf)

    return xarray_dataset


GEOSCF_HCHO_v36_2021_warm  = load_geoscf_vp_csv('2021', '05-01', '08-31')
GEOSCF_HCHO_v36_2022_warm  = load_geoscf_vp_csv('2022', '05-01', '08-31')
GEOSCF_HCHO_v36_2023_warm  = load_geoscf_vp_csv('2023', '05-01', '08-31')


In [22]:
def load_geoscf_ozone_csv(year, start_date, end_date):
    folder_path = '/import/GREENING/tzhao/GEOSCF_subset/'
    prefix = f'GEOSCF'
    suffix = f'{year}-{start_date}_{year}-{end_date}.csv'

    hcho_data = pd.read_csv(folder_path + prefix + '_O3_' + suffix, sep=',', index_col=0)
    hcho_data.index = pd.to_datetime(hcho_data.index)
    hcho_data = hcho_data.drop_duplicates().to_xarray()
    hcho_data = hcho_data.rename({'datetime_UTC': 'time'})
    
    return hcho_data
    
GEOSCF_O3_2021_warm = load_geoscf_ozone_csv('2021', '05-01', '08-31')
GEOSCF_O3_2022_warm = load_geoscf_ozone_csv('2022', '05-01', '08-31')
GEOSCF_O3_2023_warm = load_geoscf_ozone_csv('2023', '05-01', '08-31')


In [66]:
GEOSCF_HCHO_2021to2023_warm = xr.concat( [GEOSCF_HCHO_2021_warm,GEOSCF_HCHO_2022_warm,GEOSCF_HCHO_2023_warm] , dim='time')

GEOSCF_TROPCOL_HCHO_2021to2023_warm = xr.concat( [GEOSCF_TROPCOL_HCHO_2021_warm,GEOSCF_TROPCOL_HCHO_2022_warm,
                                   GEOSCF_TROPCOL_HCHO_2023_warm] , dim='time')

GEOSCF_TOTCOL_HCHO_2021to2023_warm = xr.concat( [GEOSCF_TOTCOL_HCHO_2021_warm,GEOSCF_TOTCOL_HCHO_2022_warm,
                                   GEOSCF_TOTCOL_HCHO_2023_warm] , dim='time')



In [67]:
GEOSCF_HCHO_v36_2021to2023_warm = xr.concat( [GEOSCF_HCHO_v36_2021_warm,\
                                              GEOSCF_HCHO_v36_2022_warm,\
                                              GEOSCF_HCHO_v36_2023_warm] , dim='time')

In [17]:
# load PGN data 

def fill_missing_variables(ds):

    for site in info_fush_union[:, 2]:
        # Check if the site coordinate is present in the dataset
        if site not in list(ds.variables):
            # If site is not present, create a new DataArray with NaN values
            nan_values = np.full(len(ds.time), np.nan)
            new_data_array = xr.DataArray(nan_values, coords=[ds.time], dims=['time'], name=site)
            # Concatenate the new DataArray with the original dataset along the 'site' dimension
            ds[site]=new_data_array
            
    return ds


# Define the years
years = ['2021', '2022', '2023']

# Define the filenames common part
filenames = [
    "PGN_fuh_HCHOTropVCD",
    "PGN_fuh_HCHOSurfConc",
    "PGN_fus_HCHOVCD"
]

tmp_ds_warm  = {}
tmp_ds_cold = {}

for year in years:
    for filename in filenames:

        # Construct the full filename
        full_filename = os.path.join(jndir_HCHOdiurnal_data, f"{filename}_{year}05_08.nc")
        # Open the dataset and store it in the dictionary
        tmp = xr.open_dataset(full_filename, engine="netcdf4")
        tmp = fill_missing_variables( tmp ).copy()
        tmp_ds_warm[f"{filename}_{year}_warm"] = tmp

In [19]:
PGN_fuh_HCHOTropVCD_2021to2023_warm = xr.concat([tmp_ds_warm['PGN_fuh_HCHOTropVCD_2021_warm'],\
                                                 tmp_ds_warm['PGN_fuh_HCHOTropVCD_2022_warm'],\
                                                 tmp_ds_warm['PGN_fuh_HCHOTropVCD_2023_warm']], dim='time')

PGN_fuh_HCHOSurfConc_2021to2023_warm = xr.concat([tmp_ds_warm['PGN_fuh_HCHOSurfConc_2021_warm'],\
                                                 tmp_ds_warm['PGN_fuh_HCHOSurfConc_2022_warm'],\
                                                 tmp_ds_warm['PGN_fuh_HCHOSurfConc_2023_warm']], dim='time')

PGN_fus_HCHOVCD_2021to2023_warm = xr.concat([tmp_ds_warm['PGN_fus_HCHOVCD_2021_warm'],\
                                                 tmp_ds_warm['PGN_fus_HCHOVCD_2022_warm'],\
                                                 tmp_ds_warm['PGN_fus_HCHOVCD_2023_warm']], dim='time')


In [15]:
# 修改第二维名称函数
def rename_second_dim(obj, new_dim_name):
    if len(obj.dims) > 1:
        return obj.rename({obj.dims[1]: new_dim_name})
    return obj

In [114]:
# HCHO profiles


years = ['2021', '2022', '2023']

# Define the filenames common part
filenames = [
    "PGN_Pandora_fuh_HCHOdVCDprofile",
]

tmp_ds_warm = {}
# tmp_ds_cold = {}

for year in years:
    for filename in filenames:
        # Construct the full filename
        full_filename = os.path.join(jndir_HCHOdiurnal_data, f"{filename}_{year}05_08.FINAL6.unscale.nc")
        
        # Open the dataset and store it in the dictionary
        tmp = xr.open_dataset(full_filename, engine="netcdf4")
        tmp = fill_missing_variables(tmp).copy()

        # 统一修改 data_vars 和 coords 中的第二维名称
        tmp = tmp.map(lambda v: rename_second_dim(v, "layer_center_height_bins"))

        # 删除名字以 "boxheight_binned_" 开头的 coordinates
        coords_to_drop = [coord for coord in tmp.coords if coord.startswith("layer_center_height_binned_")]
        tmp = tmp.drop_vars(coords_to_drop)

        tmp_ds_warm[f"{filename}_{year}_warm"] = tmp

In [115]:
# 获取所有数据集中出现过的坐标集合
all_coords = set()
for key in tmp_ds_warm:
    all_coords.update(tmp_ds_warm[key].coords)

# 遍历数据集，补充缺失的坐标
for key in tmp_ds_warm:
    missing_coords = all_coords - set(tmp_ds_warm[key].coords)
    for coord in missing_coords:
        # 为缺失的坐标填充 NaN
        tmp_ds_warm[key] = tmp_ds_warm[key].assign_coords({coord: np.nan})

# 合并数据集
PGN_fuh_HCHOdVCDprofile_2021to2023_warm = xr.concat(
    [tmp_ds_warm['PGN_Pandora_fuh_HCHOdVCDprofile_2021_warm'],
     tmp_ds_warm['PGN_Pandora_fuh_HCHOdVCDprofile_2022_warm'],
     tmp_ds_warm['PGN_Pandora_fuh_HCHOdVCDprofile_2023_warm']],
    dim='time'
)



In [116]:
# 需要提取的前缀类型
prefixes = [ "sfc_hourly_"]

# **1. 获取所有站点名称（即 tmp_ds_warm 的 variable name 列表）**
all_sites = set()
for year in ['2021', '2022', '2023']:
    dataset_key = f'PGN_Pandora_fuh_HCHOdVCDprofile_{year}_warm'
    ds = tmp_ds_warm[dataset_key]
    
    # 获取所有 variable names（站点名）
    all_sites.update(ds.data_vars.keys())

# **2. 创建存储不同前缀的 xarray.Dataset**
datasets_by_prefix = {prefix: {} for prefix in prefixes}

# **3. 遍历每个年份数据集，提取不同前缀的数据**
for year in ['2021', '2022', '2023']:
    dataset_key = f'PGN_Pandora_fuh_HCHOdVCDprofile_{year}_warm'
    ds = tmp_ds_warm[dataset_key]
    
    # 处理每个前缀类型
    for prefix in prefixes:
        # 提取当前前缀的坐标数据
        coord_vars = {var.replace(prefix, ""): ds.coords[var] for var in ds.coords if var.startswith(prefix)}

        # **4. 确保所有站点都存在**
        new_vars = {}
        for site in all_sites:
            if site in coord_vars:
                new_vars[site] = coord_vars[site]
            else:
                # 该站点没有该类型数据，填充 NaN
                new_vars[site] = xr.DataArray(np.full(ds.dims['time'], np.nan), dims=['time'])

        # **5. 创建新的 xarray.Dataset**
        prefix_ds = xr.Dataset(new_vars, coords={'time': ds['time']})
        
        # 存储该年前缀的数据
        datasets_by_prefix[prefix][year] = prefix_ds

# **6. 统一所有数据集的坐标**
for prefix in prefixes:
    all_coords = set()
    for year in datasets_by_prefix[prefix]:
        all_coords.update(datasets_by_prefix[prefix][year].coords)

    # 确保所有数据集具有相同的坐标
    for year in datasets_by_prefix[prefix]:
        missing_coords = all_coords - set(datasets_by_prefix[prefix][year].coords)
        for coord in missing_coords:
            datasets_by_prefix[prefix][year] = datasets_by_prefix[prefix][year].assign_coords({coord: np.nan})

# **7. 合并三年的数据**
final_datasets = {}
for prefix in prefixes:
    final_datasets[prefix] = xr.concat(
        [datasets_by_prefix[prefix]['2021'], datasets_by_prefix[prefix]['2022'], datasets_by_prefix[prefix]['2023']],
        dim='time'
    )

# **8. 结果存储**
PGN_fuh_HCHOSurfConc_2021to2023_warm = final_datasets["sfc_hourly_"]


In [22]:
def load_geoscf_met_csv(var, year, start_date, end_date):
    directory = '/import/GREENING/tzhao/GEOSCF_subset_opendap/'
    all_dataframes = []

    start_date = pd.to_datetime(f"{year}-{start_date}")
    end_date = pd.to_datetime(f"{year}-{end_date}")

    for filename in os.listdir(directory):
        if 'met_tavg_1hr_g1440x721_v36__'+ str(var) +'_merged_' + str(year) in filename and filename.endswith('.csv'):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['time'] = pd.to_datetime(df['time'])
            # Set minute, second, and microsecond to zero
            df['time'] = df['time'].dt.floor('H')  # Set to the beginning of the hour
            df = df[(df['time'] >= start_date) & (df['time'] <= end_date)]
            all_dataframes.append(df)

    concatenated_df = pd.concat(all_dataframes)
    concatenated_df = concatenated_df.drop_duplicates(subset=['time', 'lev'], keep='first')
    concatenated_df['time'] = pd.to_datetime(concatenated_df['time'])
    
    xarray_dataset = xr.Dataset.from_dataframe(concatenated_df.set_index(['time', 'lev']))
    xarray_dataset = xarray_dataset.assign_coords(lev=H_geoscf)

    return xarray_dataset



def load_geoscf_vp_met_csv(var, year, start_date, end_date):

    directory = '/import/GREENING/tzhao/GEOSCF_subset_opendap/'
    all_dataframes = []

    start_date = pd.to_datetime(f"{year}-{start_date}")
    end_date = pd.to_datetime(f"{year}-{end_date}")

    for filename in os.listdir(directory):
        if 'met_tavg_1hr_g1440x721_v36__'+ str(var) +'_merged_' + str(year) in filename and filename.endswith('.csv'):
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['time'] = pd.to_datetime(df['time'])
            # Set minute, second, and microsecond to zero
            df['time'] = df['time'].dt.floor('H')  # Set to the beginning of the hour
            df = df[(df['time'] >= start_date) & (df['time'] <= end_date)]
            all_dataframes.append(df)

    concatenated_df = pd.concat(all_dataframes)
    concatenated_df = concatenated_df.drop_duplicates(subset=['time', 'lev'], keep='first')
    concatenated_df['time'] = pd.to_datetime(concatenated_df['time'])
    
    xarray_dataset = xr.Dataset.from_dataframe(concatenated_df.set_index(['time', 'lev']))
    xarray_dataset = xarray_dataset.assign_coords(lev=H_geoscf)

    return xarray_dataset



GEOSCF_T_v36_2021_warm = load_geoscf_met_csv( 'T' ,'2021', '05-01', '08-31')
GEOSCF_T_v36_2022_warm = load_geoscf_met_csv( 'T' ,'2022', '05-01', '08-31')
GEOSCF_T_v36_2023_warm = load_geoscf_met_csv( 'T' ,'2023', '05-01', '08-31')


GEOSCF_T_v36_2021to2023_warm = xr.concat( [GEOSCF_T_v36_2021_warm,GEOSCF_T_v36_2022_warm,GEOSCF_T_v36_2023_warm] , dim='time')
GEOSCF_T_v36_2021to2023_warm = GEOSCF_T_v36_2021to2023_warm.drop_vars(['time.1', 'lev.1'])


In [96]:
#     PGN HCHO
# --------------------------------------------------
# Perform linear interpolation
from scipy.interpolate import interp1d


interp_pressure = interp1d(H_geoscf[H_geoscf<=3], P_geoscf[H_geoscf<=3], kind='linear', fill_value='extrapolate')

# Interpolate pressure for the second altitude list
layer_center_P_bins = interp_pressure(PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins)
# Select levels less than 3
tmp_below3km = GEOSCF_T_v36_2021to2023_warm.sel(lev=GEOSCF_T_v36_2021to2023_warm['lev'].where(GEOSCF_T_v36_2021to2023_warm['lev'] < 3, drop=True))

GEOSCF_T_v36_2021to2023_warm_PGNlev = tmp_below3km.interp(lev=PGN_fuh_HCHOdVCDprofile_2021to2023_warm.layer_center_height_bins.values, method='linear')

#  calculate conversion factor: 
factor_PGN_HCHOvp_ppbv_2021to2023_warm = ( 8.3145 * GEOSCF_T_v36_2021to2023_warm_PGNlev ) / (6.02e11 * layer_center_P_bins * 1e-1)


# convert unit for PGN
factor_PGN_HCHOvp_ppbv_2021to2023_warm = factor_PGN_HCHOvp_ppbv_2021to2023_warm.map(lambda v: rename_second_dim(v, "layer_center_height_bins"))
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv = PGN_fuh_HCHOdVCDprofile_2021to2023_warm * factor_PGN_HCHOvp_ppbv_2021to2023_warm
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv = PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv.map(lambda v: rename_second_dim(v, "lev"))


In [27]:
PGN_fus_HCHOVCD_2021to2023_warm_diurnal = {}
GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal = {}

PGN_fus_HCHOVCD_2021to2023_warm_diurnal_std = {}
GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_std = {}

PGN_fus_HCHOVCD_2021to2023_warm_diurnal_count = {}
GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_count = {}

PGN_fus_HCHOVCD_2021to2023_warm_diurnal_2d = {}
GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_2d = {}


for i, (lat, lon, sitename) in enumerate(info_fush_union[:,0:3]):
    
    lat = float(lat)
    lon = float(lon)


    # Select the data for the current site from the datasets
    geoscf_data = GEOSCF_TOTCOL_HCHO_2021to2023_warm[ sitename ] * 1e15

    try:
        pgn_data = PGN_fus_HCHOVCD_2021to2023_warm[ sitename ] * 6.02e19
        pgn_flag = 0  # flag=0 means PGN is not all_nan
    except:
        pgn_data = PGN_fus_HCHOVCD_2021to2023_warm[ 'FairbanksAK_P174' ].copy()
        pgn_data[~np.isnan(pgn_data)]=np.nan
        pgn_flag = 1
        
    pgn_data = pgn_data.where(np.abs(pgn_data) <= 1e50)
    geoscf_data, pgn_data = xr.align( geoscf_data, pgn_data, join='outer' )

    # -------------------------------------------
    # Align time and nan values
    nan_positions = np.isnan(pgn_data)
    # Set NaN at those positions in each variable
    pgn_data = pgn_data.where(~nan_positions, np.nan)
    geoscf_data = geoscf_data.where(~nan_positions, np.nan)
    # ------------------------------------------

    # -------------------------------------------
    # calculate the diurnal variation
    pgn_data_diurnal,pgn_data_diurnal_std,pgn_data_diurnal_count,pgn_data_diurnal_2d = calculate_diurnal_cycle_GC(pgn_data, lat, lon)[0:4]
    geoscf_data_diurnal,geoscf_data_diurnal_std,geoscf_data_diurnal_count,geoscf_data_diurnal_2d = calculate_diurnal_cycle_GC(geoscf_data, lat, lon)[0:4]
    
    PGN_fus_HCHOVCD_2021to2023_warm_diurnal[sitename] = pgn_data_diurnal
    GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal[sitename] =  geoscf_data_diurnal

    PGN_fus_HCHOVCD_2021to2023_warm_diurnal_std[sitename] = pgn_data_diurnal_std
    GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_std[sitename] =  geoscf_data_diurnal_std

    PGN_fus_HCHOVCD_2021to2023_warm_diurnal_count[sitename] = pgn_data_diurnal_count
    GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_count[sitename] = geoscf_data_diurnal_count

    PGN_fus_HCHOVCD_2021to2023_warm_diurnal_2d[sitename] = pgn_data_diurnal_2d
    GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_2d[sitename] = geoscf_data_diurnal_2d


PGN_fus_HCHOVCD_2021to2023_warm_diurnal = xr.Dataset(PGN_fus_HCHOVCD_2021to2023_warm_diurnal)
GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal = xr.Dataset(GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal)

PGN_fus_HCHOVCD_2021to2023_warm_diurnal_std = xr.Dataset(PGN_fus_HCHOVCD_2021to2023_warm_diurnal_std)
GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_std = xr.Dataset(GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_std)

PGN_fus_HCHOVCD_2021to2023_warm_diurnal_count = xr.Dataset(PGN_fus_HCHOVCD_2021to2023_warm_diurnal_count)
GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_count = xr.Dataset(GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_count)

PGN_fus_HCHOVCD_2021to2023_warm_diurnal_2d = xr.Dataset(PGN_fus_HCHOVCD_2021to2023_warm_diurnal_2d)
GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_2d = xr.Dataset(GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_2d)

In [97]:
PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal = {}
GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal = {}

PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_std = {}
GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_std = {}

PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_count = {}
GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_count = {}

PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_2d = {}
GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_2d = {}


for i, (lat, lon, sitename) in enumerate(info_fush_union[:,0:3]):
    
    lat = float(lat)
    lon = float(lon)


    # Select the data for the current site from the datasets
    geoscf_data = GEOSCF_TROPCOL_HCHO_2021to2023_warm[ sitename ]  * 1e15

    try:
        pgn_data = PGN_fuh_HCHOTropVCD_2021to2023_warm[ sitename ] * 6.02e19
        pgn_flag = 0  # flag=0 means PGN is not all_nan
    except:
        pgn_data = PGN_fuh_HCHOTropVCD_2021to2023_warm[ 'FairbanksAK_P174' ].copy()
        pgn_data[~np.isnan(pgn_data)]=np.nan
        pgn_flag = 1

    pgn_data = pgn_data.where(np.abs(pgn_data) <= 1e50)
    geoscf_data, pgn_data = xr.align( geoscf_data, pgn_data, join='outer' )

    # -------------------------------------------
    # Align time and nan values
    nan_positions = np.isnan(pgn_data)

    # Set NaN at those positions in each variable
    pgn_data = pgn_data.where(~nan_positions, np.nan)
    geoscf_data = geoscf_data.where(~nan_positions, np.nan)
    # ------------------------------------------

    # -------------------------------------------
    # calculate the diurnal variation
    pgn_data_diurnal,pgn_data_diurnal_std, pgn_data_diurnal_count, pgn_data_diurnal_2d = calculate_diurnal_cycle_GC(pgn_data, lat, lon)[0:4]
    geoscf_data_diurnal,geoscf_data_diurnal_std, geoscf_data_diurnal_count, geoscf_data_diurnal_2d = calculate_diurnal_cycle_GC(geoscf_data, lat, lon)[0:4]

    
    PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal[sitename] = pgn_data_diurnal
    GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal[sitename] =  geoscf_data_diurnal
    
    PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_std[sitename] = pgn_data_diurnal_std
    GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_std[sitename] =  geoscf_data_diurnal_std
    
    PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_count[sitename] = pgn_data_diurnal_count
    GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_count[sitename] =  geoscf_data_diurnal_count
    
    PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_2d[sitename] = pgn_data_diurnal_2d
    GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_2d[sitename] =  geoscf_data_diurnal_2d


PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal = xr.Dataset(PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal)
GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal = xr.Dataset(GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal)

PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_std = xr.Dataset(PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_std)
GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_std = xr.Dataset(GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_std)

PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_count = xr.Dataset(PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_count)
GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_count = xr.Dataset(GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_count)

PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_2d = xr.Dataset(PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_2d)
GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_2d = xr.Dataset(GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_2d)

In [117]:
PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal = {}
GEOSCF_HCHO_2021to2023_warm_diurnal = {}
PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_std = {}
GEOSCF_HCHO_2021to2023_warm_diurnal_std = {}
PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_count = {}
GEOSCF_HCHO_2021to2023_warm_diurnal_count = {}
PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_2d = {}
GEOSCF_HCHO_2021to2023_warm_diurnal_2d = {}

for i, (lat, lon, sitename) in enumerate(info_fush_union[:,0:3]):
    
    lat = float(lat)
    lon = float(lon)


    # Select the data for the current site from the datasets
    geoscf_data = GEOSCF_HCHO_2021to2023_warm[ sitename ]  * 1e9

    try:
        pgn_data = PGN_fuh_HCHOSurfConc_2021to2023_warm[ sitename ] * 22.4 / 1e3 * 1e9
        pgn_flag = 0  # flag=0 means PGN is not all_nan
    except:
        pgn_data = geoscf_data.copy()
        pgn_data[~np.isnan(pgn_data)]=np.nan
        pgn_flag = 1

    pgn_data = pgn_data.where(np.abs(pgn_data) <= 1e50)
    geoscf_data, pgn_data = xr.align( geoscf_data, pgn_data, join='outer' )
    # -------------------------------------------
    # Align time and nan values
#     if pgn_flag == 0:
    nan_positions = np.isnan(pgn_data)
    # Set NaN at those positions in each variable
    pgn_data = pgn_data.where(~nan_positions, np.nan)
    geoscf_data = geoscf_data.where(~nan_positions, np.nan)
    # -------------------------------------------

    # -------------------------------------------
    # calculate the diurnal variation
    pgn_data_diurnal,pgn_data_diurnal_std, pgn_data_diurnal_count, pgn_data_diurnal_2d = calculate_diurnal_cycle_GC(pgn_data, lat, lon)[0:4]
    geoscf_data_diurnal,geoscf_data_diurnal_std, geoscf_data_diurnal_count, geoscf_data_diurnal_2d = calculate_diurnal_cycle_GC(geoscf_data, lat, lon)[0:4]
    
    PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal[sitename] = pgn_data_diurnal
    GEOSCF_HCHO_2021to2023_warm_diurnal[sitename] =  geoscf_data_diurnal

    PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_std[sitename] = pgn_data_diurnal_std
    GEOSCF_HCHO_2021to2023_warm_diurnal_std[sitename] =  geoscf_data_diurnal_std
    
    PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_count[sitename] = pgn_data_diurnal_count
    GEOSCF_HCHO_2021to2023_warm_diurnal_count[sitename] =  geoscf_data_diurnal_count
    
    PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_2d[sitename] = pgn_data_diurnal_2d
    GEOSCF_HCHO_2021to2023_warm_diurnal_2d[sitename] =  geoscf_data_diurnal_2d


PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal = xr.Dataset(PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal)
GEOSCF_HCHO_2021to2023_warm_diurnal = xr.Dataset(GEOSCF_HCHO_2021to2023_warm_diurnal)

PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_std = xr.Dataset(PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_std)
GEOSCF_HCHO_2021to2023_warm_diurnal_std = xr.Dataset(GEOSCF_HCHO_2021to2023_warm_diurnal_std)

PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_count = xr.Dataset(PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_count)
GEOSCF_HCHO_2021to2023_warm_diurnal_count = xr.Dataset(GEOSCF_HCHO_2021to2023_warm_diurnal_count)

PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_2d = xr.Dataset(PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_2d)
GEOSCF_HCHO_2021to2023_warm_diurnal_2d = xr.Dataset(GEOSCF_HCHO_2021to2023_warm_diurnal_2d)

In [98]:
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal = {}
GEOSCF_HCHO_v36_2021to2023_warm_diurnal = {}
# GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal = {}
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_std = {}
GEOSCF_HCHO_v36_2021to2023_warm_diurnal_std = {}
# GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_std = {}
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_count = {}
GEOSCF_HCHO_v36_2021to2023_warm_diurnal_count = {}
# GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_count = {}
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_2d = {}
GEOSCF_HCHO_v36_2021to2023_warm_diurnal_2d = {}
# GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_2d = {}



for i, (lat, lon, sitename) in enumerate(info_fush_union[:,0:3]):
    
    lat = float(lat)
    lon = float(lon)

        
    # Select the data for the current site from the datasets
    try:
        geoscf_data = GEOSCF_HCHO_v36_2021to2023_warm[ sitename ]
    except:
        geoscf_data = GEOSCF_HCHO_v36_2021to2023_warm[ 'BronxNY_P180' ]
        geoscf_data[~np.isnan(geoscf_data)]=np.nan
        

    try:
        pgn_data = PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv[ sitename ]
        pgn_flag = 0  # flag=0 means PGN is not all_nan
    except:
        pgn_data = PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv[ 'BronxNY_P180' ].copy()
        pgn_data = pgn_data.where(np.isnan(pgn_data))
        pgn_flag = 1

    pgn_data = pgn_data.where(np.abs(pgn_data) <= 1e50)

    # -------------------------------------------
    # Align time and nan values
    pgn_data, geoscf_data = xr.align(pgn_data, geoscf_data, join='outer',
                                    exclude=[dim for dim in pgn_data.dims if dim != 'time' ])
    
    pgn_data_mean = np.nanmean(pgn_data, axis=1)
    nan_positions = np.where( np.isnan(pgn_data_mean) )[0]
    
    pgn_data[ nan_positions, : ] = np.nan
    geoscf_data[ nan_positions, : ] = np.nan
    
    # -------------------------------------------

    # -------------------------------------------
    # calculate the diurnal variation
    pgn_data_diurnal,pgn_data_diurnal_std, pgn_data_diurnal_count, pgn_data_diurnal_2d = calculate_diurnal_cycle_GC(pgn_data, lat, lon)[0:4]
    geoscf_data_diurnal,geoscf_data_diurnal_std, geoscf_data_diurnal_count, geoscf_data_diurnal_2d = calculate_diurnal_cycle_GC(geoscf_data, lat, lon)[0:4]
#     gc_data_diurnal,gc_data_diurnal_std, gc_data_diurnal_count = calculate_diurnal_cycle_GC(gc_data, lat, lon)[0:3]
    
    PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal[sitename] = pgn_data_diurnal
#     GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal[sitename] = gc_data_diurnal
    GEOSCF_HCHO_v36_2021to2023_warm_diurnal[sitename] =  geoscf_data_diurnal

    PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_std[sitename] = pgn_data_diurnal_std
#     GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_std[sitename] = gc_data_diurnal_std
    GEOSCF_HCHO_v36_2021to2023_warm_diurnal_std[sitename] =  geoscf_data_diurnal_std
    
    PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_count[sitename] = pgn_data_diurnal_count
#     GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_count[sitename] = gc_data_diurnal_count
    GEOSCF_HCHO_v36_2021to2023_warm_diurnal_count[sitename] =  geoscf_data_diurnal_count
    
    PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_2d[sitename] = pgn_data_diurnal_2d
#     GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_count[sitename] = gc_data_diurnal_count
    GEOSCF_HCHO_v36_2021to2023_warm_diurnal_2d[sitename] =  geoscf_data_diurnal_2d


PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal = xr.Dataset(PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal)
# GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal = xr.Dataset(GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal)
GEOSCF_HCHO_v36_2021to2023_warm_diurnal = xr.Dataset(GEOSCF_HCHO_v36_2021to2023_warm_diurnal)
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_std = xr.Dataset(PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_std)
# GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_std = xr.Dataset(GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_std)
GEOSCF_HCHO_v36_2021to2023_warm_diurnal_std = xr.Dataset(GEOSCF_HCHO_v36_2021to2023_warm_diurnal_std)
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_count = xr.Dataset(PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_count)
# GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_count = xr.Dataset(GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_count)
GEOSCF_HCHO_v36_2021to2023_warm_diurnal_count = xr.Dataset(GEOSCF_HCHO_v36_2021to2023_warm_diurnal_count)
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_2d = xr.Dataset(PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_2d)
# GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_2d = xr.Dataset(GC_SpeciesConc_CH2O_2021to2023_warm_vp_diurnal_2d)
GEOSCF_HCHO_v36_2021to2023_warm_diurnal_2d = xr.Dataset(GEOSCF_HCHO_v36_2021to2023_warm_diurnal_2d)

missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day
missed 1 day

In [71]:
PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal = {}
PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal_std = {}
PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal_count = {}
PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal_2d = {}

for i, (lat, lon, sitename) in enumerate(info_fush_union[:,0:3]):
    
    lat = float(lat)
    lon = float(lon)

    # Select the data for the current site from the datasets
    geoscf_data = GEOSCF_HCHO_2021to2023_warm[ sitename ]

    try:
        pgn_data = PGN_fuh_profile_count_raw_df_2021to2023_warm[ sitename ]
        pgn_flag = 0  # flag=0 means PGN is not all_nan
    except:
        pgn_data = geoscf_data.copy()
        pgn_data[~np.isnan(pgn_data)]=np.nan
        pgn_flag = 1

    pgn_data = pgn_data.where(np.abs(pgn_data) <= 1e50)
    geoscf_data, pgn_data = xr.align( geoscf_data, pgn_data, join='outer' )
    # -------------------------------------------
    # Align time and nan values
#     if pgn_flag == 0:
    nan_positions = np.isnan(pgn_data)
    # Set NaN at those positions in each variable
    pgn_data = pgn_data.where(~nan_positions, np.nan)
    # -------------------------------------------

    # -------------------------------------------
    # calculate the diurnal variation
    pgn_data_diurnal,pgn_data_diurnal_std, pgn_data_diurnal_count, pgn_data_diurnal_2d = calculate_diurnal_cycle_GC(pgn_data, lat, lon)[0:4]
     
    PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal[sitename] = pgn_data_diurnal
    PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal_2d[sitename] = pgn_data_diurnal_2d


PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal = xr.Dataset(  PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal  )
PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal_2d = xr.Dataset(  PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal_2d  )

In [72]:
PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal = {}
PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal_std = {}
PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal_count = {}
PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal_2d = {}

for i, (lat, lon, sitename) in enumerate(info_fush_union[:,0:3]):
    
    lat = float(lat)
    lon = float(lon)

    # Select the data for the current site from the datasets
    geoscf_data = GEOSCF_HCHO_2021to2023_warm[ sitename ]

    try:
        pgn_data = PGN_fuh_profile_count_QC1_df_2021to2023_warm[ sitename ]
        pgn_flag = 0  # flag=0 means PGN is not all_nan
    except:
        pgn_data = geoscf_data.copy()
        pgn_data[~np.isnan(pgn_data)]=np.nan
        pgn_flag = 1

    pgn_data = pgn_data.where(np.abs(pgn_data) <= 1e50)
    geoscf_data, pgn_data = xr.align( geoscf_data, pgn_data, join='outer' )
    # -------------------------------------------
    # Align time and nan values
#     if pgn_flag == 0:
    nan_positions = np.isnan(pgn_data)
    # Set NaN at those positions in each variable
    pgn_data = pgn_data.where(~nan_positions, np.nan)
    # -------------------------------------------

    # -------------------------------------------
    # calculate the diurnal variation
    pgn_data_diurnal,pgn_data_diurnal_std, pgn_data_diurnal_count, pgn_data_diurnal_2d = calculate_diurnal_cycle_GC(pgn_data, lat, lon)[0:4]
     
    PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal[sitename] = pgn_data_diurnal
    PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal_2d[sitename] = pgn_data_diurnal_2d


PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal = xr.Dataset(  PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal  )
PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal_2d = xr.Dataset(  PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal_2d  )

In [99]:
PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal = {}
PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal_std = {}
PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal_count = {}
PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal_2d = {}

for i, (lat, lon, sitename) in enumerate(info_fush_union[:,0:3]):
    
    lat = float(lat)
    lon = float(lon)

    # Select the data for the current site from the datasets
    geoscf_data = GEOSCF_HCHO_2021to2023_warm[ sitename ]

    try:
        pgn_data = PGN_fuh_profile_count_QC2_df_2021to2023_warm[ sitename ]
        pgn_flag = 0  # flag=0 means PGN is not all_nan
    except:
        pgn_data = geoscf_data.copy()
        pgn_data[~np.isnan(pgn_data)]=np.nan
        pgn_flag = 1

    pgn_data = pgn_data.where(np.abs(pgn_data) <= 1e50)
    geoscf_data, pgn_data = xr.align( geoscf_data, pgn_data, join='outer' )
    # -------------------------------------------
    # Align time and nan values
#     if pgn_flag == 0:
    nan_positions = np.isnan(pgn_data)
    # Set NaN at those positions in each variable
    pgn_data = pgn_data.where(~nan_positions, np.nan)
    # -------------------------------------------

    # -------------------------------------------
    # calculate the diurnal variation
    pgn_data_diurnal,pgn_data_diurnal_std, pgn_data_diurnal_count, pgn_data_diurnal_2d = calculate_diurnal_cycle_GC(pgn_data, lat, lon)[0:4]
     
    PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal[sitename] = pgn_data_diurnal
    PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal_2d[sitename] = pgn_data_diurnal_2d


PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal = xr.Dataset(  PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal  )
PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal_2d = xr.Dataset(  PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal_2d  )

In [100]:
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal = {}
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_std = {}
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_count = {}
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_2d = {}



for i, (lat, lon, sitename) in enumerate(info_fush_union[:,0:3]):
    
    lat = float(lat)
    lon = float(lon)

        
    # Select the data for the current site from the datasets
    try:
        geoscf_data = GEOSCF_HCHO_v36_2021to2023_warm[ sitename ]
    except:
        geoscf_data = GEOSCF_HCHO_v36_2021to2023_warm[ 'BronxNY_P180' ]
        geoscf_data[~np.isnan(geoscf_data)]=np.nan
        

    try:
        pgn_data = PGN_fuh_HCHOdVCDprofile_2021to2023_warm[ sitename ] * 250 *1e2
        pgn_flag = 0  # flag=0 means PGN is not all_nan
    except:
        pgn_data = PGN_fuh_HCHOdVCDprofile_2021to2023_warm[ 'BronxNY_P180' ].copy() * 250 *1e2
        pgn_data = pgn_data.where(np.isnan(pgn_data))
        pgn_flag = 1

    pgn_data = pgn_data.where(np.abs(pgn_data) <= 1e50)

    # -------------------------------------------
    # Align time and nan values
    pgn_data, geoscf_data = xr.align(pgn_data, geoscf_data, join='outer',
                                    exclude=[dim for dim in pgn_data.dims if dim != 'time' ])
    try:
        pgn_data_mean = np.nanmean(pgn_data, axis=1)
        nan_positions = np.where( np.isnan(pgn_data_mean) )[0]
        
        pgn_data[ nan_positions, : ] = np.nan
        geoscf_data[ nan_positions, : ] = np.nan

    except:
        pgn_data = geoscf_data.copy()
        pgn_data = pgn_data.where(np.isnan(pgn_data))
        
    
    # -------------------------------------------

    # -------------------------------------------
    # calculate the diurnal variation
    pgn_data_diurnal,pgn_data_diurnal_std, pgn_data_diurnal_count, pgn_data_diurnal_2d = calculate_diurnal_cycle_GC(pgn_data, lat, lon)[0:4]
    
    PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal[sitename] = pgn_data_diurnal
    PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_std[sitename] = pgn_data_diurnal_std
    PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_count[sitename] = pgn_data_diurnal_count
    PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_2d[sitename] = pgn_data_diurnal_2d


PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal = xr.Dataset(PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal)
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_std = xr.Dataset(PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_std)
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_count = xr.Dataset(PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_count)
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_2d = xr.Dataset(PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_2d)

In [101]:
def replace_lev_vars_with_nan_and_adjust_dims(dataset):
    """
    找出所有依赖 'lev' 这个坐标的变量：
    - 用全 NaN 替换它们
    - 重新调整它们的坐标结构，使其与不含 'lev' 的变量一致
    - 最后删除 'lev' 坐标
    
    Parameters:
        dataset (xr.Dataset): 需要处理的 xarray 数据集
    
    Returns:
        xr.Dataset: 处理后的数据集
    """
    # 复制 dataset，避免修改原数据
    dataset = dataset.copy()
    
    # 找出所有使用 'lev' 作为维度的变量
    vars_with_lev = [var for var in dataset.data_vars if 'lev' in dataset[var].dims]

    # 获取 dataset 中其他变量的主要坐标维度
    other_vars = [var for var in dataset.data_vars if var not in vars_with_lev]
    if other_vars:
        reference_dims = dataset[other_vars[0]].dims  # 选一个没有 'lev' 的变量作为参考
    else:
        reference_dims = tuple(dim for dim in dataset.dims if dim != "lev")  # 如果所有变量都有 lev

    # 替换这些变量为全 NaN，并调整维度
    for var in vars_with_lev:
        shape = tuple(dataset.sizes[dim] for dim in reference_dims)  # 参考其他变量的 shape
        dataset[var] = (reference_dims, np.full(shape, np.nan))  # 赋值为全 NaN 并修改维度

    # 删除 'lev' 这个坐标（如果还存在）
    if 'lev' in dataset.coords:
        dataset = dataset.drop_vars('lev')

    return dataset


PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal = replace_lev_vars_with_nan_and_adjust_dims(  PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal)
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_std = replace_lev_vars_with_nan_and_adjust_dims(  PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_std)
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_count = replace_lev_vars_with_nan_and_adjust_dims(  PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_count)
PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_2d = replace_lev_vars_with_nan_and_adjust_dims(  PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_2d)

In [120]:
# Export to netcdf files for the processed diurnal data 
processed_variable_names = [
    # "GEOSCF_HCHO_2021to2023_warm",
    # "GEOSCF_TROPCOL_HCHO_2021to2023_warm",
    # "GEOSCF_TOTCOL_HCHO_2021to2023_warm",
    # # "GEOSCF_HCHO_v36_2021to2023_warm",
    # "PGN_fuh_HCHOTropVCD_2021to2023_warm",
    # "PGN_fuh_HCHOSurfConc_2021to2023_warm",
    # # "PGN_fuh_HCHOdVCDprofile_2021to2023_warm",
    # "PGN_fus_HCHOVCD_2021to2023_warm",
    # # "GEOSCF_T_v36_2021to2023_warm",
    # # "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv",
    # # "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv",
    
    # "PGN_fus_HCHOVCD_2021to2023_warm_diurnal",
    # "GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal",
    # "PGN_fus_HCHOVCD_2021to2023_warm_diurnal_std",
    # "GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_std",
    # "PGN_fus_HCHOVCD_2021to2023_warm_diurnal_count",
    # "GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_count",
    # "PGN_fus_HCHOVCD_2021to2023_warm_diurnal_2d",
    # "GEOSCF_TOTCOL_HCHO_2021to2023_warm_diurnal_2d",

    "PGN_fuh_HCHOTropVCD_2021to2023_warm",
    "GEOSCF_TROPCOL_HCHO_2021to2023_warm",
    "PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal",
    "GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal",
    "PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_std",
    "GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_std",
    "PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_count",
    "GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_count",
    "PGN_fuh_HCHOTropVCD_2021to2023_warm_diurnal_2d",
    "GEOSCF_TROPCOL_HCHO_2021to2023_warm_diurnal_2d",

    # "PGN_fuh_HCHOSurfConc_2021to2023_warm",
    # "GEOSCF_HCHO_2021to2023_warm",
    # "PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal",
    # "GEOSCF_HCHO_2021to2023_warm_diurnal",
    # "PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_std",
    # "GEOSCF_HCHO_2021to2023_warm_diurnal_std",
    # "PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_count",
    # "GEOSCF_HCHO_2021to2023_warm_diurnal_count",
    # "PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_2d",
    # "GEOSCF_HCHO_2021to2023_warm_diurnal_2d",

    # "GEOSCF_HCHO_v36_2021to2023_warm",
    # "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv",
    # "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal",
    # "GEOSCF_HCHO_v36_2021to2023_warm_diurnal",
    # "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_std",
    # "GEOSCF_HCHO_v36_2021to2023_warm_diurnal_std",
    # "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_count",
    # "GEOSCF_HCHO_v36_2021to2023_warm_diurnal_count",
    # "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_2d",
    # "GEOSCF_HCHO_v36_2021to2023_warm_diurnal_2d",

]


# processed_variable_names = [

#     "GEOSCF_HCHO_v36_2021to2023_warm",
#     "PGN_fuh_HCHOdVCDprofile_2021to2023_warm",
#     # "GEOSCF_T_v36_2021to2023_warm",
#     "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv",
#     "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal",
#     "GEOSCF_HCHO_v36_2021to2023_warm_diurnal",
#     "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_std",
#     "GEOSCF_HCHO_v36_2021to2023_warm_diurnal_std",
#     "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_count",
#     "GEOSCF_HCHO_v36_2021to2023_warm_diurnal_count",
#     "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_ppbv_diurnal_2d",
#     "GEOSCF_HCHO_v36_2021to2023_warm_diurnal_2d",

# ]


processed_variable_names = [
    "PGN_fuh_HCHOSurfConc_2021to2023_warm",
    "GEOSCF_HCHO_2021to2023_warm",
    "PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal",
    "GEOSCF_HCHO_2021to2023_warm_diurnal",
    "PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_std",
    "GEOSCF_HCHO_2021to2023_warm_diurnal_std",
    "PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_count",
    "GEOSCF_HCHO_2021to2023_warm_diurnal_count",
    "PGN_fuh_HCHOSurfConc_2021to2023_warm_diurnal_2d",
    "GEOSCF_HCHO_2021to2023_warm_diurnal_2d",
]


# processed_variable_names = [
#     "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal",
#     "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_std",
#     "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_count",
#     "PGN_fuh_HCHOdVCDprofile_2021to2023_warm_dVCD_diurnal_2d",
# ]


processed_variable_names = [
    "PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal",
    "PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal_2d",
    "PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal",
    "PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal_2d",
    "PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal",
    "PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal_2d",
]


In [121]:
# Loop over each variable name
for varname in processed_variable_names:
    # Access the variable directly from memory
    data = locals()[varname]

    # Save the dataset or data array to a .nc file
    # Replace `path_to_save` with the directory where you want to save the .nc files
    data.to_netcdf(f"/import/GREENING/tzhao/jndata/paper4_data/{varname}.nc", engine='netcdf4')
    print(f"Saved to /import/GREENING/tzhao/jndata/paper4_data/{varname}.nc")


Saved to /import/GREENING/tzhao/jndata/paper4_data/PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal.nc
Saved to /import/GREENING/tzhao/jndata/paper4_data/PGN_fuh_profile_count_raw_df_2021to2023_warm_diurnal_2d.nc
Saved to /import/GREENING/tzhao/jndata/paper4_data/PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal.nc
Saved to /import/GREENING/tzhao/jndata/paper4_data/PGN_fuh_profile_count_QC1_df_2021to2023_warm_diurnal_2d.nc
Saved to /import/GREENING/tzhao/jndata/paper4_data/PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal.nc
Saved to /import/GREENING/tzhao/jndata/paper4_data/PGN_fuh_profile_count_QC2_df_2021to2023_warm_diurnal_2d.nc
